In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Код из библиотеки dlnlputils репозитория https://github.com/Samsung-IT-Academy/stepik-dl-nlp

In [ ]:
#########stepik-dl-nlp/dlnlputils/data/base.py#########

import collections
import re

import numpy as np

TOKEN_RE = re.compile(r'[\w\d]+')


def tokenize_text_simple_regex(txt, min_token_size=4):
    txt = txt.lower()
    all_tokens = TOKEN_RE.findall(txt)
    return [token for token in all_tokens if len(token) >= min_token_size]


def tokenize_corpus(texts, tokenizer=tokenize_text_simple_regex, **tokenizer_kwargs):
    return [tokenizer(text, **tokenizer_kwargs) for text in texts]

def tokenize_corpus_verbose(texts, tokenizer=tokenize_text_simple_regex, **tokenizer_kwargs):
    tokenize_texts = []
    for i, text in enumerate(texts):
        tokenize_texts.append(tokenizer(text, **tokenizer_kwargs))
        if i % 1000 == 0:
            print('Complete:', i)
    return tokenize_texts


def build_vocabulary(tokenized_texts, max_size=1000000, max_doc_freq=0.8, min_count=5, pad_word=None):
    word_counts = collections.defaultdict(int)
    doc_n = 0

    # посчитать количество документов, в которых употребляется каждое слово
    # а также общее количество документов
    for txt in tokenized_texts:
        doc_n += 1
        unique_text_tokens = set(txt)
        for token in unique_text_tokens:
            word_counts[token] += 1

    # убрать слишком редкие и слишком частые слова
    word_counts = {word: cnt for word, cnt in word_counts.items()
                   if cnt >= min_count and cnt / doc_n <= max_doc_freq}

    # отсортировать слова по убыванию частоты
    sorted_word_counts = sorted(word_counts.items(),
                                reverse=True,
                                key=lambda pair: pair[1])

    # добавим несуществующее слово с индексом 0 для удобства пакетной обработки
    if pad_word is not None:
        sorted_word_counts = [(pad_word, 0)] + sorted_word_counts

    # если у нас по прежнему слишком много слов, оставить только max_size самых частотных
    if len(word_counts) > max_size:
        sorted_word_counts = sorted_word_counts[:max_size]

    # нумеруем слова
    word2id = {word: i for i, (word, _) in enumerate(sorted_word_counts)}

    # нормируем частоты слов
    word2freq = np.array([cnt / doc_n for _, cnt in sorted_word_counts], dtype='float32')

    return word2id, word2freq



#########stepik-dl-nlp/dlnlputils/data/bag_of_words.py#########

import numpy as np
import scipy.sparse
import torch
from torch.utils.data import Dataset


def vectorize_texts(tokenized_texts, word2id, word2freq, mode='tfidf', scale=True):
    #modified by me 
    #add 'lftidf', 'tflidf', 'ltflidf', 'ltf', 'lidf'
    
    assert mode in {'tfidf', 'idf', 'tf', 'bin', 'ltfidf', 'tflidf', 'tflidf_v2', 'ltf', 'tfpmi'}

    # считаем количество употреблений каждого слова в каждом документе
    result = scipy.sparse.dok_matrix((len(tokenized_texts), len(word2id)), dtype='float32')
    for text_i, text in enumerate(tokenized_texts):
        for token in text:
            if token in word2id:
                result[text_i, word2id[token]] += 1

    # получаем бинарные вектора "встречается или нет"
    if mode == 'bin':
        result = (result > 0).astype('float32')

    # получаем вектора относительных частот слова в документе
    elif mode == 'tf':
        result = result.tocsr()
        result = result.multiply(1 / result.sum(1))

    # полностью убираем информацию о количестве употреблений слова в данном документе,
    # но оставляем информацию о частотности слова в корпусе в целом
    elif mode == 'idf':
        result = (result > 0).astype('float32').multiply(1 / word2freq)

    # учитываем всю информацию, которая у нас есть:
    # частоту слова в документе и частоту слова в корпусе
    elif mode == 'tfidf':
        result = result.tocsr()
        result = result.multiply(1 / result.sum(1))  # разделить каждую строку на её длину
        result = result.multiply(1 / word2freq)  # разделить каждый столбец на вес слова

    elif mode == 'ltf': # lTF=ln⁡(TF+1)
        result = result.tocsr()
        result = result.multiply(1 / result.sum(1))
        result = scipy.sparse.dok_matrix(np.log(result.toarray()+1))
 
    elif mode == 'lidf': # lIDF=ln⁡(n/IDF+1)
        result = (result > 0).astype('float32').multiply(len(tokenized_texts) / word2freq)
        result = scipy.sparse.dok_matrix(np.log(result.toarray()+1))

        
    elif mode == 'ltfidf': # lTFIDF=ln⁡(TF+1)⋅IDF
        result = result.tocsr() #переводим матрицу в режим быстрой работы со строками (это очень важно!!)
        result = result.multiply(1/result.sum(1)) # разделить каждую строку на её длину
        result = scipy.sparse.dok_matrix(np.log(result.toarray()+1))
        result = result.multiply(1 / word2freq) # разделить каждый столбец на вес слова
        

    elif mode == 'tflidf': # lTFIDF=TF⋅ln⁡(1/IDF+1)
        result = result.tocsr() #переводим матрицу в режим быстрой работы со строками (это очень важно!!)
        result = result.multiply(1/result.sum(1)) # разделить каждую строку на её длину
        result = result.multiply(np.log(1 / word2freq + 1)) # разделить каждый столбец на вес слова

    elif mode == 'tflidf_v2': # lTFIDF=TF⋅ln⁡(n/IDF+1)
        result = result.tocsr() #переводим матрицу в режим быстрой работы со строками (это очень важно!!)
        result = result.multiply(1/result.sum(1)) # разделить каждую строку на её длину
        result = result.multiply(np.log(len(tokenized_texts) / word2freq + 1)) # разделить каждый столбец на вес слова
        
    elif mode == 'tfpmi': # TFPMI=TF⋅PMI
        result = result.tocsr()
        result = result.multiply(1 / result.sum(1))  # разделить каждую строку на её длину
        result = result.multiply(word2freq)  # домножить каждую строку на word2freq (это массив PMI Scores)

    if scale:
        result = result.tocsc()
        result -= result.min()
        result /= (result.max() + 1e-6)

    return result.tocsr()


class SparseFeaturesDataset(Dataset):
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets

    def __len__(self):
        return self.features.shape[0]

    def __getitem__(self, idx):
        cur_features = torch.from_numpy(self.features[idx].toarray()[0]).float()
        cur_label = torch.from_numpy(np.asarray(self.targets[idx])).long()
        return cur_features, cur_label
    
    
#########stepik-dl-nlp/dlnlputils/pipeline.py#########

import copy
import datetime
import random
import traceback

import numpy as np
import torch
from torch.utils.data import DataLoader


def init_random_seed(value=0):
    random.seed(value)
    np.random.seed(value)
    torch.manual_seed(value)
    torch.cuda.manual_seed(value)
    torch.backends.cudnn.deterministic = True


def copy_data_to_device(data, device):
    if torch.is_tensor(data):
        return data.to(device)
    elif isinstance(data, (list, tuple)):
        return [copy_data_to_device(elem, device) for elem in data]
    raise ValueError('Недопустимый тип данных {}'.format(type(data)))


def print_grad_stats(model):
    mean = 0
    std = 0
    norm = 1e-5
    for param in model.parameters():
        grad = getattr(param, 'grad', None)
        if grad is not None:
            mean += grad.data.abs().mean()
            std += grad.data.std()
            norm += 1
    mean /= norm
    std /= norm
    print(f'Mean grad {mean}, std {std}, n {norm}')


def train_eval_loop(model, train_dataset, val_dataset, criterion,
                    lr=1e-4, epoch_n=10, batch_size=32,
                    device=None, early_stopping_patience=10, l2_reg_alpha=0,
                    max_batches_per_epoch_train=10000,
                    max_batches_per_epoch_val=1000,
                    data_loader_ctor=DataLoader,
                    optimizer_ctor=None,
                    lr_scheduler_ctor=None,
                    shuffle_train=True,
                    dataloader_workers_n=0,
                    best_acc_type = 'loss',
                    test_dataset = None,
                    experiment_name = 'NoName'):
    """
    Цикл для обучения модели. После каждой эпохи качество модели оценивается по отложенной выборке.
    :param model: torch.nn.Module - обучаемая модель
    :param train_dataset: torch.utils.data.Dataset - данные для обучения
    :param val_dataset: torch.utils.data.Dataset - данные для оценки качества
    :param criterion: функция потерь для настройки модели
    :param lr: скорость обучения
    :param epoch_n: максимальное количество эпох
    :param batch_size: количество примеров, обрабатываемых моделью за одну итерацию
    :param device: cuda/cpu - устройство, на котором выполнять вычисления
    :param early_stopping_patience: наибольшее количество эпох, в течение которых допускается
        отсутствие улучшения модели, чтобы обучение продолжалось.
    :param l2_reg_alpha: коэффициент L2-регуляризации
    :param max_batches_per_epoch_train: максимальное количество итераций на одну эпоху обучения
    :param max_batches_per_epoch_val: максимальное количество итераций на одну эпоху валидации
    :param data_loader_ctor: функция для создания объекта, преобразующего датасет в батчи
        (по умолчанию torch.utils.data.DataLoader)
    :return: кортеж из двух элементов:
        - среднее значение функции потерь на валидации на лучшей эпохе
        - лучшая модель
    """
    
    '''
    modified by wisoffe
    best_acc_type: 'loss' or 'acc'
    experiment_name: 
    '''
    assert best_acc_type in {'loss', 'acc'}
    
    train_start_time = datetime.datetime.now()
    print("############## Start experiment with name: {} ##############".format(experiment_name))
    
    #statistics history
    history = {'acc': {'train': [0.0],
                       'val': [0.0]},
               'loss': {'train': [float('inf')],
                       'val': [float('inf')]}}
    
    if device is None:
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
    device = torch.device(device)
    model.to(device)

    if optimizer_ctor is None:
        optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=l2_reg_alpha)
    else:
        optimizer = optimizer_ctor(model.parameters(), lr=lr)

    if lr_scheduler_ctor is not None:
        lr_scheduler = lr_scheduler_ctor(optimizer)
    else:
        lr_scheduler = None

    train_dataloader = data_loader_ctor(train_dataset, batch_size=batch_size, shuffle=shuffle_train,
                                        num_workers=dataloader_workers_n)
    val_dataloader = data_loader_ctor(val_dataset, batch_size=batch_size, shuffle=False,
                                      num_workers=dataloader_workers_n)
    
    if best_acc_type == 'loss': #отбираем модель по минимальному loss
        best_val_metric = float('inf')
    elif best_acc_type == 'acc': #отбираем модель по максимальному accuracy
        best_val_metric = float('-inf')
        
    best_epoch_i = 0
    best_model = copy.deepcopy(model)
    
    
    for epoch_i in range(1, epoch_n + 1):
        try:
            #####train phase######
            epoch_start = datetime.datetime.now()
            train_accuracy_epoch = [] #for statistics
            train_loss_epoch = [] #for statistics
            
            model.train()
            
            for batch_i, (batch_x, batch_y) in enumerate(train_dataloader):
                if batch_i > max_batches_per_epoch_train:
                    print('Threshold max_batches_per_epoch_train exceeded!')
                    break

                batch_x = copy_data_to_device(batch_x, device)
                batch_y = copy_data_to_device(batch_y, device)

                pred = model(batch_x)
                loss = criterion(pred, batch_y)

                model.zero_grad()
                loss.backward()

                optimizer.step()

                train_loss_epoch.append(float(loss))
                
                train_accuracy_epoch.append(float((pred.argmax(dim=1) == batch_y.data).float().mean().data))
                #train_accuracy_epoch.append(float((pred.detach().cpu().numpy().argmax(-1) == batch_y.detach().cpu().numpy()).mean()))
            
            #####validation phase######
            model.eval()

            val_accuracy_epoch = [] #for statistics
            val_loss_epoch = [] #for statistics

            with torch.no_grad():
                for batch_i, (batch_x, batch_y) in enumerate(val_dataloader):
                    if batch_i > max_batches_per_epoch_val:
                        print('Threshold max_batches_per_epoch_val exceeded!')
                        break

                    batch_x = copy_data_to_device(batch_x, device)
                    batch_y = copy_data_to_device(batch_y, device)

                    pred = model(batch_x)
                    loss = criterion(pred, batch_y)
                    
                    val_accuracy_epoch.append(float((pred.argmax(dim=1) == batch_y.data).float().mean().data))
                    #val_accuracy_epoch.append(float((pred.detach().cpu().numpy().argmax(-1) == batch_y.detach().cpu().numpy()).mean()))
                    val_loss_epoch.append(float(loss))

            
            ########ending of epoch#########
            
            history['acc']['train'].append(sum(train_accuracy_epoch) / len(train_accuracy_epoch))
            history['loss']['train'].append(sum(train_loss_epoch) / len(train_loss_epoch))  

            history['acc']['val'].append(sum(val_accuracy_epoch) / len(val_accuracy_epoch))
            history['loss']['val'].append(sum(val_loss_epoch) / len(val_loss_epoch))
            
            
            #save best model
            best_model_saved = False
            if (best_acc_type == 'loss' and history['loss']['val'][-1] < best_val_metric) or \
                    (best_acc_type == 'acc' and history['acc']['val'][-1] > best_val_metric):
                #отбираем модель по минимальному loss или максимальному accuracy
                best_epoch_i = epoch_i
                best_val_metric = history[best_acc_type]['val'][-1]
                best_model = copy.deepcopy(model)
                best_model_saved = True
            #check for break training
            elif epoch_i - best_epoch_i > early_stopping_patience:
                print('Модель не улучшилась за последние {} эпох, прекращаем обучение'.format(
                    early_stopping_patience))
                break

            if lr_scheduler is not None:
                lr_scheduler.step(history['loss']['val'][-1])
            
            #output statistics
            
            print('Epoch = {:>3},   ACC: val = {:.3f}, train = {:.3f}    LOSS: val = {:.3f}, train = {:.3f}   SAVE: {}, Time: {:0.2f}s'\
                  .format(epoch_i,
                          history['acc']['val'][-1], 
                          history['acc']['train'][-1],
                          history['loss']['val'][-1],
                          history['loss']['train'][-1],
                          best_model_saved,
                          (datetime.datetime.now() - epoch_start).total_seconds()),
                  flush=True)

        except KeyboardInterrupt:
            print('Досрочно остановлено пользователем')
            break
        except Exception as ex:
            print('Ошибка при обучении: {}\n{}'.format(ex, traceback.format_exc()))
            break
            
    print(' ')
    print("BEST MODEL: ACC: val = {:.3f}, train = {:.3f}, LOSS: val = {:.3f}, train = {:.3f}, on epoch = {}, metric type = {}, Full train time = {:0.2f}s"\
                  .format(history['acc']['val'][best_epoch_i], 
                          history['acc']['train'][best_epoch_i],
                          history['loss']['val'][best_epoch_i],
                          history['loss']['train'][best_epoch_i],
                          best_epoch_i,
                          best_acc_type,
                          (datetime.datetime.now() - train_start_time).total_seconds()))
    print("************** End experiment with name: {} **************".format(experiment_name))
    print(' ')
    history['BEST'] = {}
    history['BEST']['epoch'] = best_epoch_i
    history['BEST']['dict_size'] = batch_x.shape[-1]
    
    
    #calculate and save final metrics best_model on train/val/test datasets
    if test_dataset is not None:
        history['BEST']['acc'] = {}
        history['BEST']['loss'] = {}
        
        #save validation metrics (no calculate again)
        history['BEST']['acc']['val'] = history['acc']['val'][best_epoch_i]
        history['BEST']['loss']['val'] = history['loss']['val'][best_epoch_i]
        
        #calculate and save train metrics
        train_pred = predict_with_model(best_model, train_dataset, return_labels=True)
        history['BEST']['loss']['train'] = float(F.cross_entropy(torch.from_numpy(train_pred[0]),
                             torch.from_numpy(train_pred[1]).long()))
        history['BEST']['acc']['train'] = accuracy_score(train_pred[1], train_pred[0].argmax(-1))
        
        #calculate and save test metrics
        test_pred = predict_with_model(best_model, test_dataset, return_labels=True)
        history['BEST']['loss']['test'] = float(F.cross_entropy(torch.from_numpy(test_pred[0]),
                             torch.from_numpy(test_pred[1]).long()))
        history['BEST']['acc']['test'] = accuracy_score(test_pred[1], test_pred[0].argmax(-1))    
    
    
    return history, best_model


def predict_with_model(model, dataset, device=None, batch_size=32, num_workers=0, return_labels=False):
    """
    :param model: torch.nn.Module - обученная модель
    :param dataset: torch.utils.data.Dataset - данные для применения модели
    :param device: cuda/cpu - устройство, на котором выполнять вычисления
    :param batch_size: количество примеров, обрабатываемых моделью за одну итерацию
    :return: numpy.array размерности len(dataset) x *
    """
    if device is None:
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
    results_by_batch = []

    device = torch.device(device)
    model.to(device)
    model.eval()

    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
    labels = []
    with torch.no_grad():
        import tqdm
        for batch_x, batch_y in tqdm.tqdm(dataloader, total=len(dataset)/batch_size):
            batch_x = copy_data_to_device(batch_x, device)

            if return_labels:
                labels.append(batch_y.numpy())

            batch_pred = model(batch_x)
            results_by_batch.append(batch_pred.detach().cpu().numpy())

    if return_labels:
        return np.concatenate(results_by_batch, 0), np.concatenate(labels, 0)
    else:
        return np.concatenate(results_by_batch, 0)

### Мои наработки

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import spacy
spacy_nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

def tokenize_text_spacy_lemmatize(txt, spacy_nlp, min_token_size=1):
    doc = spacy_nlp(txt)
    return [token.lemma_ for token in doc if len(token) >= min_token_size]

def tokenize_corpus_convert(tokenized_corpus, converter, addition = False):
    '''
    Convert each token in tokenized_corpus by converter
    
    Sample (PorterStemmer):
    import nltk
    ps = nltk.stemmer.PorterStemmer()
    tokenized_stemmed_corpus = tokenize_corpus_convert(tokenized_corpus, converter=ps.stem)
    
    Sample (SnowballStemmer):
    import nltk
    sno = nltk.stem.SnowballStemmer('english')
    tokenized_stemmed_corpus = tokenize_corpus_convert(tokenized_corpus, converter=sno.stem)
    
    Sample (WordNetLemmatizer):
    import nltk
    lemma = nltk.wordnet.WordNetLemmatizer()
    tokenized_lemmas_corpus = tokenize_corpus_convert(tokenized_corpus, converter=lemma.lemmatize)
    '''
    output = []
    if not addition: #возвращаем только преобразованные токены
        for doc in tokenized_corpus:
            output.append([converter(token) for token in doc])
    else: #возвращаем списк из исходных токенов, дополненных списком преобразованных
        for doc in tokenized_corpus:
            output.append(doc + [converter(token) for token in doc])        
    return output

def show_experiments_stats(histories, figsize = (16.0, 6.0), show_plots = True, only_BEST_MODEL_CALC = False):
    matplotlib.rcParams['figure.figsize'] = figsize
    
    for experiment_id in histories.keys():
        print('{:-<100}'.format(experiment_id))
        
        if not only_BEST_MODEL_CALC:
            epoch_max_acc = np.array(histories[experiment_id]['acc']['val']).argmax()
            print('Max val acc on:    Epoch = {:>3},   ACCURACY: val  = {:.3f}, train = {:.3f},   LOSS: val  = {:.3f}, train = {:.3f}'\
                  .format(epoch_max_acc, 
                          histories[experiment_id]['acc']['val'][epoch_max_acc], 
                          histories[experiment_id]['acc']['train'][epoch_max_acc],
                          histories[experiment_id]['loss']['val'][epoch_max_acc],
                          histories[experiment_id]['loss']['train'][epoch_max_acc]))
            epoch_min_loss = np.array(histories[experiment_id]['loss']['val']).argmin()
            print('Min val loss on:   Epoch = {:>3},   ACCURACY: val  = {:.3f}, train = {:.3f},   LOSS: val  = {:.3f}, train = {:.3f}'\
                  .format(epoch_min_loss, 
                          histories[experiment_id]['acc']['val'][epoch_min_loss], 
                          histories[experiment_id]['acc']['train'][epoch_min_loss],
                          histories[experiment_id]['loss']['val'][epoch_min_loss],
                          histories[experiment_id]['loss']['train'][epoch_min_loss]))
        
        if 'acc' in histories[experiment_id]['BEST']:
            print("BEST MODEL CALC:   Epoch = {:>3},   ACCURACY: test = {:.3f}, train = {:.3f},   LOSS: test = {:.3f}, train = {:.3f}  DICT SIZE = {}"\
                  .format(histories[experiment_id]['BEST']['epoch'], 
                          histories[experiment_id]['BEST']['acc']['test'],
                          histories[experiment_id]['BEST']['acc']['train'],
                          histories[experiment_id]['BEST']['loss']['test'],
                          histories[experiment_id]['BEST']['loss']['train'],
                          histories[experiment_id]['BEST']['dict_size']))
    
    
    if show_plots:
        for experiment_id in histories.keys():
            plt.plot(histories[experiment_id]['acc']['val'], label=experiment_id + ' val')
        plt.legend()
        plt.title('Validation Accuracy (Val only)')
        plt.show()

        for experiment_id in histories.keys():
            plt.plot(histories[experiment_id]['acc']['val'], label=experiment_id + ' val')
            plt.plot(histories[experiment_id]['acc']['train'], label=experiment_id + ' train')
        plt.legend()
        plt.title('Validation Accuracy (Val/Train)');
        plt.show()

        for experiment_id in histories.keys():
            plt.plot(histories[experiment_id]['loss']['val'], label=experiment_id  + ' val')
        plt.legend()
        plt.title('Validation Loss (Val only)');
        plt.show()

        for experiment_id in histories.keys():
            plt.plot(histories[experiment_id]['loss']['val'], label=experiment_id  + ' val')
            plt.plot(histories[experiment_id]['loss']['train'], label=experiment_id  + ' train')
        plt.legend()
        plt.title('Validation Loss (Val/Train)');
        plt.show()
        

def build_vocabulary_pmi(tokenized_texts, texts_label_indexes, max_size=1000000, 
                         invers_quantile_threshold = 1.0, min_count = 5, 
                         pmi_type = 'max', no_negative = False, 
                         p_w_l_denominator = 'size(l)', pad_word=None):
    '''
    texts_label_indexes: содержит индексы (не названия) меток, соответсвующие документам из tokenized_texts
    '''
    
    assert pmi_type in {'max', 'mean'}
    assert p_w_l_denominator in {'size(l)', 'size(c)'}
    
    #формируем массив маржинальных вероятностей встретить документ класса label
    # p(l) = Size(l)/Size(c)
    _, counts_labels = np.unique(texts_label_indexes, return_counts=True)
    p_labels = counts_labels / len(texts_label_indexes)
    
    tokens_list = list(set([item for sublist in tokenized_texts for item in sublist]))
    
    tokens_dict_indexes = {k: v for v, k in enumerate(tokens_list)}
    
    #для подсчета количества документов, в которых встречается токен
    token_counts_corpus = np.zeros(len(tokens_list))
    #для подсчета количества документов класса label, в которых встречается токен
    token_counts_labels = np.zeros((len(tokens_list), len(p_labels)))
    
    # посчитываем количество документов (общее и распеределенное по меткам), 
    # в которых употребляется каждое слово
    for doc_i, doc in enumerate(tokenized_texts):
        unique_doc_tokens = set(doc)
        for token in unique_doc_tokens:
            token_counts_corpus[tokens_dict_indexes[token]] += 1
            token_counts_labels[tokens_dict_indexes[token]][texts_label_indexes[doc_i]] += 1
    
    #оставляем только слова повторяющиеся >= min_count
    mask = token_counts_corpus >= min_count
    token_counts_corpus = token_counts_corpus[mask]
    token_counts_labels = token_counts_labels[mask]
    tokens_list = list(np.array(tokens_list)[mask])
    
    #массив маржинальных вероятностей употребления слов в корпусе
    # p(w) = DocCount(w,c)/Size(c)
    p_words = token_counts_corpus / len(tokenized_texts)
    
    if p_w_l_denominator == 'size(l)':
        #массив вероятностей встретить слово в документе класса label
        # p(w,l) = DocCount(w,l)/Size(l)
        p_words_labels = token_counts_labels / counts_labels
    elif p_w_l_denominator == 'size(c)':
        #массив совместных вероятностей w и l во всем корпусе
        # p(w,l) = DocCount(w,l)/len(tokenized_texts)
        p_words_labels = token_counts_labels / counts_labels
    
    #Pointwise mutal information words in labels
    # pmi(l,w) = log(p(w,l)/(p(w)*p(l)))
    pmi_words_labels = p_words_labels / p_labels
    pmi_words_labels = (pmi_words_labels.T / p_words).T
    eps = 1e-10
    pmi_words_labels = np.log(pmi_words_labels + eps)
    pmi_words_labels[pmi_words_labels < 0] = 0 #replace negative by 0

    if not no_negative:
        #массив маржинальных вероятностей НЕ встретить слово в корпусе
        # p(~w) = DocCount(~w,c)/Size(c)
        p_words_negative = 1.0 - p_words

        #массив вероятностей НЕ встретить слово в документе класса label
        # p(~w,l) = DocCount(~w,l)/Size(l)
        p_words_labels_negative = 1.0 - p_words_labels

        #Pointwise mutal information NO words in labels
        # pmi(l,~w) = log(p(~w,l)/(p(~w)*p(l)))
        pmi_words_labels_negative = p_words_labels_negative / p_labels
        pmi_words_labels_negative = (pmi_words_labels_negative.T / p_words_negative).T
        pmi_words_labels_negative = np.log(pmi_words_labels_negative + eps)
        pmi_words_labels_negative[pmi_words_labels_negative < 0] = 0 #replace negative by 0 
        if pmi_type == 'max':
            pmi_word_scores = pmi_words_labels.max(axis=1) + p_words_labels_negative.max(axis=1)
        elif pmi_type == 'mean':
            pmi_word_scores = pmi_words_labels.mean(axis=1) + p_words_labels_negative.mean(axis=1)
    else:
        if pmi_type == 'max':
            pmi_word_scores = pmi_words_labels.max(axis=1)
        elif pmi_type == 'mean':
            pmi_word_scores = pmi_words_labels.mean(axis=1)
    
    if invers_quantile_threshold < 1.0:
        invers_quantile = np.quantile(pmi_word_scores, 1.0 - invers_quantile_threshold)
        mask = pmi_word_scores > invers_quantile
        pmi_word_scores = pmi_word_scores[mask]
        tokens_list = list(np.array(tokens_list)[mask])
    
    pmi_word_dict = {k: v for k, v in zip(tokens_list, list(pmi_word_scores))}
    
    #отсортировать слова по убыванию pmi scores
    sorted_word_counts = sorted(pmi_word_dict.items(),
                                 reverse=True,
                                 key=lambda pair: pair[1])
    
    # добавим несуществующее слово с индексом 0 для удобства пакетной обработки
    if pad_word is not None:
        sorted_word_counts = [(pad_word, 0)] + sorted_word_counts

    # если у нас по прежнему слишком много слов, оставить только max_size с самым большим pmi score
    if len(sorted_word_counts) > max_size:
        sorted_word_counts = sorted_word_counts[:max_size]
        
    # нумеруем слова
    word2id = {word: i for i, (word, _) in enumerate(sorted_word_counts)}

    # формируем итоговый массив pmi score
    word2pmi_score = np.array([score for _, score in sorted_word_counts], dtype='float32')

    return word2id, word2pmi_score

        
#https://stackoverflow.com/questions/4529815/saving-an-object-data-persistence/4529901
import pickle
def save_object(obj, filename):
    with open(filename, 'wb') as outp:  # Overwrites any existing file.
        pickle.dump(obj, outp, pickle.HIGHEST_PROTOCOL)

def load_object(filename):
    with open(filename, 'rb') as inp:
        return pickle.load(inp)

# sample usage
#company1 = [1,2,3,4,5]
#save_object(company1, '/kaggle/working/company1.pkl')
#del company
#company1 = load_object(filename)

## Загрузка датасета

In [ ]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.metrics import accuracy_score

train_source = fetch_20newsgroups(subset='train')
test_source = fetch_20newsgroups(subset='test')

print('Количество обучающих текстов', len(train_source['data']))
print('Количество тестовых текстов', len(test_source['data']))
print()
print(train_source['data'][0].strip())

print()
print('Метка', train_source['target'][0])

In [ ]:
train_source.keys()

In [ ]:
train_source['target_names']

## Реализация на scikit-learn

In [ ]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.pipeline import Pipeline
# from sklearn.linear_model import LogisticRegression
# import torch
# from torch import nn
# from torch.nn import functional as F

In [ ]:
# MAX_DF = 0.8
# MIN_COUNT = 5

# sklearn_pipeline = Pipeline((('vect', TfidfVectorizer(tokenizer=tokenize_text_simple_regex,
#                                                       max_df=MAX_DF,
#                                                       min_df=MIN_COUNT)),
#                              ('cls', LogisticRegression())))
# sklearn_pipeline.fit(train_source['data'], train_source['target']);

In [ ]:
# #Оценка качества
# sklearn_train_pred = sklearn_pipeline.predict_proba(train_source['data'])
# sklearn_train_loss = F.cross_entropy(torch.from_numpy(sklearn_train_pred),
#                                                  torch.from_numpy(train_source['target']))
# print('Среднее значение функции потерь на обучении', float(sklearn_train_loss))
# print('Доля верных ответов', accuracy_score(train_source['target'], sklearn_train_pred.argmax(-1)))
# print()

# sklearn_test_pred = sklearn_pipeline.predict_proba(test_source['data'])
# sklearn_test_loss = F.cross_entropy(torch.from_numpy(sklearn_test_pred),
#                                                 torch.from_numpy(test_source['target']))
# print('Среднее значение функции потерь на валидации', float(sklearn_test_loss))
# print('Доля верных ответов', accuracy_score(test_source['target'], sklearn_test_pred.argmax(-1)))

## Реализация из семинара, которую необходимо улучшить до уровня scikit-learn или выше

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from sklearn.datasets import fetch_20newsgroups
from sklearn.metrics import accuracy_score

import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import collections

import torch
from torch import nn
from torch.nn import functional as F

histories = {}
best_models = {}

### Подготовка признаков

In [ ]:
# train_tokenized = tokenize_corpus(train_source['data'])
# test_tokenized = tokenize_corpus(test_source['data'])

# print(' '.join(train_tokenized[0]))

In [ ]:
# MAX_DF = 0.8
# MIN_COUNT = 5
# vocabulary, word_doc_freq = build_vocabulary(train_tokenized, max_doc_freq=MAX_DF, min_count=MIN_COUNT)
# UNIQUE_WORDS_N = len(vocabulary)
# print('Количество уникальных токенов', UNIQUE_WORDS_N)
# print(list(vocabulary.items())[:10])

In [ ]:
# word_doc_freq[:20]

In [ ]:
# word_doc_freq[-20:]

In [ ]:
# plt.hist(word_doc_freq, bins=20)
# plt.title('Распределение относительных частот слов')
# plt.yscale('log');

In [ ]:
# VECTORIZATION_MODE = 'tfidf'
# train_vectors = vectorize_texts(train_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)
# test_vectors = vectorize_texts(test_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)

# print('Размерность матрицы признаков обучающей выборки', train_vectors.shape)
# print('Размерность матрицы признаков тестовой выборки', test_vectors.shape)
# print()
# print('Количество ненулевых элементов в обучающей выборке', train_vectors.nnz)
# print('Процент заполненности матрицы признаков {:.2f}%'.format(train_vectors.nnz * 100 / (train_vectors.shape[0] * train_vectors.shape[1])))
# print()
# print('Количество ненулевых элементов в тестовой выборке', test_vectors.nnz)
# print('Процент заполненности матрицы признаков {:.2f}%'.format(test_vectors.nnz * 100 / (test_vectors.shape[0] * test_vectors.shape[1])))

In [ ]:
# plt.hist(train_vectors.data, bins=20)
# plt.title('Распределение весов признаков')
# plt.yscale('log');

### Распределение классов

In [ ]:
UNIQUE_LABELS_N = len(set(train_source['target']))
print('Количество уникальных меток', UNIQUE_LABELS_N)
set(train_source['target'])

In [ ]:
# plt.hist(train_source['target'], bins=np.arange(0, 21))
# plt.title('Распределение меток в обучающей выборке');

In [ ]:
# plt.hist(test_source['target'], bins=np.arange(0, 21))
# plt.title('Распределение меток в тестовой выборке');

### PyTorch Dataset

In [ ]:
# train_dataset = SparseFeaturesDataset(train_vectors, train_source['target'])
# test_dataset = SparseFeaturesDataset(test_vectors, test_source['target'])

### Обучение модели на PyTorch

In [ ]:
# model = nn.Linear(UNIQUE_WORDS_N, UNIQUE_LABELS_N)

# scheduler = lambda optim: \
#     torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=5, factor=0.5, verbose=True)

# history, best_epoch_i, best_model = train_eval_loop(model=model,
#                                             train_dataset=train_dataset,
#                                             val_dataset=test_dataset,
#                                             criterion=F.cross_entropy,
#                                             lr=1e-1,
#                                             epoch_n=200,
#                                             batch_size=32,
#                                             l2_reg_alpha=0,
#                                             lr_scheduler_ctor=scheduler,
#                                             test_dataset=test_dataset,
#                                             best_acc_type = 'loss')

### Оценка качества

In [ ]:
# train_pred = predict_with_model(best_model, train_dataset)

# train_loss = F.cross_entropy(torch.from_numpy(train_pred),
#                              torch.from_numpy(train_source['target']).long())

# print('Среднее значение функции потерь на обучении', float(train_loss))
# print('Доля верных ответов', accuracy_score(train_source['target'], train_pred.argmax(-1)))
# print()



# test_pred = predict_with_model(best_model, test_dataset)

# test_loss = F.cross_entropy(torch.from_numpy(test_pred),
#                             torch.from_numpy(test_source['target']).long())

# print('Среднее значение функции потерь на валидации', float(test_loss))
# print('Доля верных ответов', accuracy_score(test_source['target'], test_pred.argmax(-1)))

## Домашнее задание 2.4.Final

**Варианты с регуляризацией и разным lr**

In [ ]:
# model = nn.Linear(UNIQUE_WORDS_N, UNIQUE_LABELS_N)

# scheduler = lambda optim: \
#     torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=5, factor=0.5, verbose=True)

# history, best_epoch_i, best_model = train_eval_loop(model=model,
#                                             train_dataset=train_dataset,
#                                             val_dataset=test_dataset,
#                                             criterion=F.cross_entropy,
#                                             lr=1e-1,
#                                             epoch_n=200,
#                                             batch_size=32,
#                                             l2_reg_alpha=0,
#                                             lr_scheduler_ctor=scheduler,
#                                             test_dataset=test_dataset,
#                                             best_acc_type = 'acc')

### Оценка качества

In [ ]:
# train_pred = predict_with_model(best_model, train_dataset)

# train_loss = F.cross_entropy(torch.from_numpy(train_pred),
#                              torch.from_numpy(train_source['target']).long())

# print('Среднее значение функции потерь на обучении', float(train_loss))
# print('Доля верных ответов', accuracy_score(train_source['target'], train_pred.argmax(-1)))
# print()



# test_pred = predict_with_model(best_model, test_dataset)

# test_loss = F.cross_entropy(torch.from_numpy(test_pred),
#                             torch.from_numpy(test_source['target']).long())

# print('Среднее значение функции потерь на валидации', float(test_loss))
# print('Доля верных ответов', accuracy_score(test_source['target'], test_pred.argmax(-1)))

**Варианты с разными подходами по взвешиванию**

In [ ]:
# VECTORIZATION_MODE = 'tflidf'
# train_vectors = vectorize_texts(train_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)
# test_vectors = vectorize_texts(test_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)

# print('Размерность матрицы признаков обучающей выборки', train_vectors.shape)
# print('Размерность матрицы признаков тестовой выборки', test_vectors.shape)
# print()
# print('Количество ненулевых элементов в обучающей выборке', train_vectors.nnz)
# print('Процент заполненности матрицы признаков {:.2f}%'.format(train_vectors.nnz * 100 / (train_vectors.shape[0] * train_vectors.shape[1])))
# print()
# print('Количество ненулевых элементов в тестовой выборке', test_vectors.nnz)
# print('Процент заполненности матрицы признаков {:.2f}%'.format(test_vectors.nnz * 100 / (test_vectors.shape[0] * test_vectors.shape[1])))

In [ ]:
# train_dataset = SparseFeaturesDataset(train_vectors, train_source['target'])
# test_dataset = SparseFeaturesDataset(test_vectors, test_source['target'])

In [ ]:
# model = nn.Linear(UNIQUE_WORDS_N, UNIQUE_LABELS_N)

# scheduler = lambda optim: \
#     torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=5, factor=0.5, verbose=True)

# experiment_name = 'tflidf_lr=1e-2'
# histories[experiment_name], best_models[experiment_name] = train_eval_loop(
#                                             model=model,
#                                             train_dataset=train_dataset,
#                                             val_dataset=test_dataset,
#                                             criterion=F.cross_entropy,
#                                             lr=1e-2,
#                                             epoch_n=200,
#                                             batch_size=32,
#                                             l2_reg_alpha=0,
#                                             lr_scheduler_ctor=scheduler,
#                                             best_acc_type = 'acc',
#                                             test_dataset=test_dataset,
#                                             experiment_name = experiment_name)

In [ ]:
# show_experiments_stats(histories, show_plots = False)

### Эксперименты со стеммингом и лемматизацией

In [ ]:
# %%time

# ####load saved or generate tokenize lemmatization (by spacy)####
# import os.path

# filename = 'train_tokenized_lemmas.pkl'
# if os.path.isfile('/kaggle/working/'+filename):
#     train_tokenized_lemmas = load_object('/kaggle/working/'+filename)
# else:
#     train_tokenized_lemmas = tokenize_corpus_verbose(train_source['data'], tokenizer=tokenize_text_spacy_lemmatize, 
#                                           spacy_nlp=spacy_nlp, min_token_size=2)
#     save_object(train_tokenized_lemmas, '/kaggle/working/'+filename)

# filename = 'test_tokenized_lemmas.pkl'
# if os.path.isfile('/kaggle/working/'+filename):
#     test_tokenized_lemmas = load_object('/kaggle/working/'+filename)
# else:
#     test_tokenized_lemmas = tokenize_corpus_verbose(train_source['data'], tokenizer=tokenize_text_spacy_lemmatize, 
#                                           spacy_nlp=spacy_nlp, min_token_size=2)
#     save_object(test_tokenized_lemmas, '/kaggle/working/'+filename)


# print(' '.join(train_tokenized_lemmas[0]))

In [ ]:
# train_tokenized = tokenize_corpus(train_source['data'])
# test_tokenized = tokenize_corpus(test_source['data'])

# print(' '.join(train_tokenized[0]))

In [ ]:
# %%time

# # #PorterStemmer:
# # from nltk.stem.porter import PorterStemmer
# # ps = PorterStemmer()
# # converter = ps.stem

# # #SnowballStemmer:
# # from nltk.stem.snowball import SnowballStemmer
# # sno = SnowballStemmer('english')
# # converter = sno.stem
    
# #WordNetLemmatizer:
# from nltk.stem import WordNetLemmatizer
# lemma = WordNetLemmatizer()
# converter=lemma.lemmatize


# train_tokenized = tokenize_corpus_convert(train_tokenized, converter=converter, addition = True)
# test_tokenized = tokenize_corpus_convert(test_tokenized, converter=converter, addition = True)

# print(' '.join(train_tokenized[0]))

In [ ]:
# MAX_DF = 0.8
# MIN_COUNT = 5
# vocabulary, word_doc_freq = build_vocabulary(train_tokenized, max_doc_freq=MAX_DF, min_count=MIN_COUNT)
# UNIQUE_WORDS_N = len(vocabulary)
# print('Количество уникальных токенов', UNIQUE_WORDS_N)
# print(list(vocabulary.items())[:10])

In [ ]:
# word_doc_freq[:20]

In [ ]:
# word_doc_freq[-20:]

In [ ]:
# plt.hist(word_doc_freq, bins=20)
# plt.title('Распределение относительных частот слов')
# plt.yscale('log');

In [ ]:
# VECTORIZATION_MODE = 'tfidf'
# train_vectors = vectorize_texts(train_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)
# test_vectors = vectorize_texts(test_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)

# print('Размерность матрицы признаков обучающей выборки', train_vectors.shape)
# print('Размерность матрицы признаков тестовой выборки', test_vectors.shape)
# print()
# print('Количество ненулевых элементов в обучающей выборке', train_vectors.nnz)
# print('Процент заполненности матрицы признаков {:.2f}%'.format(train_vectors.nnz * 100 / (train_vectors.shape[0] * train_vectors.shape[1])))
# print()
# print('Количество ненулевых элементов в тестовой выборке', test_vectors.nnz)
# print('Процент заполненности матрицы признаков {:.2f}%'.format(test_vectors.nnz * 100 / (test_vectors.shape[0] * test_vectors.shape[1])))

In [ ]:
# plt.hist(train_vectors.data, bins=20)
# plt.title('Распределение весов признаков')
# plt.yscale('log');

### Распределение классов

In [ ]:
# UNIQUE_LABELS_N = len(set(train_source['target']))
# print('Количество уникальных меток', UNIQUE_LABELS_N)
# set(train_source['target'])

In [ ]:
# plt.hist(train_source['target'], bins=np.arange(0, 21))
# plt.title('Распределение меток в обучающей выборке');

In [ ]:
# plt.hist(test_source['target'], bins=np.arange(0, 21))
# plt.title('Распределение меток в тестовой выборке');

### PyTorch Dataset

In [ ]:
# train_dataset = SparseFeaturesDataset(train_vectors, train_source['target'])
# test_dataset = SparseFeaturesDataset(test_vectors, test_source['target'])

In [ ]:
# model = nn.Linear(UNIQUE_WORDS_N, UNIQUE_LABELS_N)

# scheduler = lambda optim: \
#     torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=5, factor=0.5, verbose=True)

# experiment_name = 'add_lemm_or_stem__default'
# histories[experiment_name], best_models[experiment_name] = train_eval_loop(
#                                             model=model,
#                                             train_dataset=train_dataset,
#                                             val_dataset=test_dataset,
#                                             criterion=F.cross_entropy,
#                                             lr=1e-1,
#                                             epoch_n=200,
#                                             batch_size=32,
#                                             l2_reg_alpha=0,
#                                             lr_scheduler_ctor=scheduler,
#                                             best_acc_type = 'acc',
#                                             test_dataset=test_dataset,
#                                             experiment_name = experiment_name)

In [ ]:
# VECTORIZATION_MODE = 'tf'
# train_vectors = vectorize_texts(train_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)
# test_vectors = vectorize_texts(test_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)

# print('Размерность матрицы признаков обучающей выборки', train_vectors.shape)
# print('Размерность матрицы признаков тестовой выборки', test_vectors.shape)
# print()
# print('Количество ненулевых элементов в обучающей выборке', train_vectors.nnz)
# print('Процент заполненности матрицы признаков {:.2f}%'.format(train_vectors.nnz * 100 / (train_vectors.shape[0] * train_vectors.shape[1])))
# print()
# print('Количество ненулевых элементов в тестовой выборке', test_vectors.nnz)
# print('Процент заполненности матрицы признаков {:.2f}%'.format(test_vectors.nnz * 100 / (test_vectors.shape[0] * test_vectors.shape[1])))

In [ ]:
# model = nn.Linear(UNIQUE_WORDS_N, UNIQUE_LABELS_N)

# scheduler = lambda optim: \
#     torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=5, factor=0.5, verbose=True)

# experiment_name = 'add_lemm_or_stem__tf_lr=1e-2'
# histories[experiment_name], best_models[experiment_name] = train_eval_loop(
#                                             model=model,
#                                             train_dataset=train_dataset,
#                                             val_dataset=test_dataset,
#                                             criterion=F.cross_entropy,
#                                             lr=1e-2,
#                                             epoch_n=200,
#                                             batch_size=32,
#                                             l2_reg_alpha=0,
#                                             lr_scheduler_ctor=scheduler,
#                                             best_acc_type = 'acc',
#                                             test_dataset=test_dataset,
#                                             experiment_name = experiment_name)

In [ ]:
# VECTORIZATION_MODE = 'tflidf'
# train_vectors = vectorize_texts(train_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)
# test_vectors = vectorize_texts(test_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)

# print('Размерность матрицы признаков обучающей выборки', train_vectors.shape)
# print('Размерность матрицы признаков тестовой выборки', test_vectors.shape)
# print()
# print('Количество ненулевых элементов в обучающей выборке', train_vectors.nnz)
# print('Процент заполненности матрицы признаков {:.2f}%'.format(train_vectors.nnz * 100 / (train_vectors.shape[0] * train_vectors.shape[1])))
# print()
# print('Количество ненулевых элементов в тестовой выборке', test_vectors.nnz)
# print('Процент заполненности матрицы признаков {:.2f}%'.format(test_vectors.nnz * 100 / (test_vectors.shape[0] * test_vectors.shape[1])))

In [ ]:
# train_dataset = SparseFeaturesDataset(train_vectors, train_source['target'])
# test_dataset = SparseFeaturesDataset(test_vectors, test_source['target'])

In [ ]:
# model = nn.Linear(UNIQUE_WORDS_N, UNIQUE_LABELS_N)

# scheduler = lambda optim: \
#     torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=5, factor=0.5, verbose=True)

# experiment_name = 'add_lemm_or_stem__tflidf_lr=1e-2'
# histories[experiment_name], best_models[experiment_name] = train_eval_loop(
#                                             model=model,
#                                             train_dataset=train_dataset,
#                                             val_dataset=test_dataset,
#                                             criterion=F.cross_entropy,
#                                             lr=1e-2,
#                                             epoch_n=200,
#                                             batch_size=32,
#                                             l2_reg_alpha=0,
#                                             lr_scheduler_ctor=scheduler,
#                                             best_acc_type = 'acc',
#                                             test_dataset=test_dataset,
#                                             experiment_name = experiment_name)

In [ ]:
# show_experiments_stats(histories, show_plots = False)

### Эксперимент - не убирать токены менее 4 символов при токенизации (параметр пминимальная длина токенов)

In [ ]:
# train_tokenized = tokenize_corpus(train_source['data'], min_token_size=2)
# test_tokenized = tokenize_corpus(test_source['data'], min_token_size=2)

# print(' '.join(train_tokenized[0]))

In [ ]:
# MAX_DF = 0.8
# MIN_COUNT = 5
# vocabulary, word_doc_freq = build_vocabulary(train_tokenized, max_doc_freq=MAX_DF, min_count=MIN_COUNT)
# UNIQUE_WORDS_N = len(vocabulary)
# print('Количество уникальных токенов', UNIQUE_WORDS_N)
# print(list(vocabulary.items())[:10])

In [ ]:
# word_doc_freq[:20]

In [ ]:
# word_doc_freq[-20:]

In [ ]:
# plt.hist(word_doc_freq, bins=20)
# plt.title('Распределение относительных частот слов')
# plt.yscale('log');

In [ ]:
# VECTORIZATION_MODE = 'tfidf'
# train_vectors = vectorize_texts(train_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)
# test_vectors = vectorize_texts(test_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)

# print('Размерность матрицы признаков обучающей выборки', train_vectors.shape)
# print('Размерность матрицы признаков тестовой выборки', test_vectors.shape)
# print()
# print('Количество ненулевых элементов в обучающей выборке', train_vectors.nnz)
# print('Процент заполненности матрицы признаков {:.2f}%'.format(train_vectors.nnz * 100 / (train_vectors.shape[0] * train_vectors.shape[1])))
# print()
# print('Количество ненулевых элементов в тестовой выборке', test_vectors.nnz)
# print('Процент заполненности матрицы признаков {:.2f}%'.format(test_vectors.nnz * 100 / (test_vectors.shape[0] * test_vectors.shape[1])))

In [ ]:
# plt.hist(train_vectors.data, bins=20)
# plt.title('Распределение весов признаков')
# plt.yscale('log');

### Распределение классов

In [ ]:
# UNIQUE_LABELS_N = len(set(train_source['target']))
# print('Количество уникальных меток', UNIQUE_LABELS_N)
# set(train_source['target'])

In [ ]:
# plt.hist(train_source['target'], bins=np.arange(0, 21))
# plt.title('Распределение меток в обучающей выборке');

In [ ]:
# plt.hist(test_source['target'], bins=np.arange(0, 21))
# plt.title('Распределение меток в тестовой выборке');

### PyTorch Dataset

In [ ]:
# train_dataset = SparseFeaturesDataset(train_vectors, train_source['target'])
# test_dataset = SparseFeaturesDataset(test_vectors, test_source['target'])

In [ ]:
# model = nn.Linear(UNIQUE_WORDS_N, UNIQUE_LABELS_N)

# scheduler = lambda optim: \
#     torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=5, factor=0.5, verbose=True)

# experiment_name = 'min_token_size=2_default'
# histories[experiment_name], best_models[experiment_name] = train_eval_loop(
#                                             model=model,
#                                             train_dataset=train_dataset,
#                                             val_dataset=test_dataset,
#                                             criterion=F.cross_entropy,
#                                             lr=1e-1,
#                                             epoch_n=200,
#                                             batch_size=32,
#                                             l2_reg_alpha=0,
#                                             lr_scheduler_ctor=scheduler,
#                                             best_acc_type = 'acc',
#                                             test_dataset=test_dataset,
#                                             experiment_name = experiment_name)

In [ ]:
# VECTORIZATION_MODE = 'tf'
# train_vectors = vectorize_texts(train_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)
# test_vectors = vectorize_texts(test_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)

# print('Размерность матрицы признаков обучающей выборки', train_vectors.shape)
# print('Размерность матрицы признаков тестовой выборки', test_vectors.shape)
# print()
# print('Количество ненулевых элементов в обучающей выборке', train_vectors.nnz)
# print('Процент заполненности матрицы признаков {:.2f}%'.format(train_vectors.nnz * 100 / (train_vectors.shape[0] * train_vectors.shape[1])))
# print()
# print('Количество ненулевых элементов в тестовой выборке', test_vectors.nnz)
# print('Процент заполненности матрицы признаков {:.2f}%'.format(test_vectors.nnz * 100 / (test_vectors.shape[0] * test_vectors.shape[1])))

In [ ]:
# model = nn.Linear(UNIQUE_WORDS_N, UNIQUE_LABELS_N)

# scheduler = lambda optim: \
#     torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=5, factor=0.5, verbose=True)

# experiment_name = 'min_token_size=2_tf_lr=1e-2'
# histories[experiment_name], best_models[experiment_name] = train_eval_loop(
#                                             model=model,
#                                             train_dataset=train_dataset,
#                                             val_dataset=test_dataset,
#                                             criterion=F.cross_entropy,
#                                             lr=1e-2,
#                                             epoch_n=200,
#                                             batch_size=32,
#                                             l2_reg_alpha=0,
#                                             lr_scheduler_ctor=scheduler,
#                                             best_acc_type = 'acc',
#                                             test_dataset=test_dataset,
#                                             experiment_name = experiment_name)

In [ ]:
# VECTORIZATION_MODE = 'tflidf'
# train_vectors = vectorize_texts(train_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)
# test_vectors = vectorize_texts(test_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)

# print('Размерность матрицы признаков обучающей выборки', train_vectors.shape)
# print('Размерность матрицы признаков тестовой выборки', test_vectors.shape)
# print()
# print('Количество ненулевых элементов в обучающей выборке', train_vectors.nnz)
# print('Процент заполненности матрицы признаков {:.2f}%'.format(train_vectors.nnz * 100 / (train_vectors.shape[0] * train_vectors.shape[1])))
# print()
# print('Количество ненулевых элементов в тестовой выборке', test_vectors.nnz)
# print('Процент заполненности матрицы признаков {:.2f}%'.format(test_vectors.nnz * 100 / (test_vectors.shape[0] * test_vectors.shape[1])))

In [ ]:
# train_dataset = SparseFeaturesDataset(train_vectors, train_source['target'])
# test_dataset = SparseFeaturesDataset(test_vectors, test_source['target'])

In [ ]:
# model = nn.Linear(UNIQUE_WORDS_N, UNIQUE_LABELS_N)

# scheduler = lambda optim: \
#     torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=5, factor=0.5, verbose=True)

# experiment_name = 'min_token_size=2_tflidf_lr=1e-2'
# histories[experiment_name], best_models[experiment_name] = train_eval_loop(
#                                             model=model,
#                                             train_dataset=train_dataset,
#                                             val_dataset=test_dataset,
#                                             criterion=F.cross_entropy,
#                                             lr=1e-2,
#                                             epoch_n=200,
#                                             batch_size=32,
#                                             l2_reg_alpha=0,
#                                             lr_scheduler_ctor=scheduler,
#                                             best_acc_type = 'acc',
#                                             test_dataset=test_dataset,
#                                             experiment_name = experiment_name)

In [ ]:
# show_experiments_stats(histories, show_plots = False)

### Эксперименты с разной длиной словаря (пороги отсечения наиболее частых и редких токенов)

In [ ]:
# train_tokenized = tokenize_corpus(train_source['data'], min_token_size=2)
# test_tokenized = tokenize_corpus(test_source['data'], min_token_size=2)

# print(' '.join(train_tokenized[0]))

In [ ]:
# MAX_DF = 0.95
# MIN_COUNT = 5
# vocabulary, word_doc_freq = build_vocabulary(train_tokenized, max_doc_freq=MAX_DF, min_count=MIN_COUNT)
# UNIQUE_WORDS_N = len(vocabulary)
# print('Количество уникальных токенов', UNIQUE_WORDS_N)
# print(list(vocabulary.items())[:10])

# # word_doc_freq[:20]

# # word_doc_freq[-20:]

# # plt.hist(word_doc_freq, bins=20)
# # plt.title('Распределение относительных частот слов')
# # plt.yscale('log');

# VECTORIZATION_MODE = 'tfidf'
# train_vectors = vectorize_texts(train_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)
# test_vectors = vectorize_texts(test_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)

# print('Размерность матрицы признаков обучающей выборки', train_vectors.shape)
# print('Размерность матрицы признаков тестовой выборки', test_vectors.shape)
# print()
# print('Количество ненулевых элементов в обучающей выборке', train_vectors.nnz)
# print('Процент заполненности матрицы признаков {:.2f}%'.format(train_vectors.nnz * 100 / (train_vectors.shape[0] * train_vectors.shape[1])))
# print()
# print('Количество ненулевых элементов в тестовой выборке', test_vectors.nnz)
# print('Процент заполненности матрицы признаков {:.2f}%'.format(test_vectors.nnz * 100 / (test_vectors.shape[0] * test_vectors.shape[1])))

# # plt.hist(train_vectors.data, bins=20)
# # plt.title('Распределение весов признаков')
# # plt.yscale('log');

# UNIQUE_LABELS_N = len(set(train_source['target']))
# print('Количество уникальных меток', UNIQUE_LABELS_N)
# set(train_source['target'])

# # plt.hist(train_source['target'], bins=np.arange(0, 21))
# # plt.title('Распределение меток в обучающей выборке');

# # plt.hist(test_source['target'], bins=np.arange(0, 21))
# # plt.title('Распределение меток в тестовой выборке');

# train_dataset = SparseFeaturesDataset(train_vectors, train_source['target'])
# test_dataset = SparseFeaturesDataset(test_vectors, test_source['target'])

# model = nn.Linear(UNIQUE_WORDS_N, UNIQUE_LABELS_N)

# scheduler = lambda optim: \
#     torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=5, factor=0.5, verbose=True)

# experiment_name = 'MAX_DF=0.95_min_token_size=2_default'
# histories[experiment_name], best_models[experiment_name] = train_eval_loop(
#                                             model=model,
#                                             train_dataset=train_dataset,
#                                             val_dataset=test_dataset,
#                                             criterion=F.cross_entropy,
#                                             lr=1e-1,
#                                             epoch_n=200,
#                                             batch_size=32,
#                                             l2_reg_alpha=0,
#                                             lr_scheduler_ctor=scheduler,
#                                             best_acc_type = 'acc',
#                                             test_dataset=test_dataset,
#                                             experiment_name = experiment_name)

# VECTORIZATION_MODE = 'tf'
# train_vectors = vectorize_texts(train_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)
# test_vectors = vectorize_texts(test_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)

# # print('Размерность матрицы признаков обучающей выборки', train_vectors.shape)
# # print('Размерность матрицы признаков тестовой выборки', test_vectors.shape)
# # print()
# # print('Количество ненулевых элементов в обучающей выборке', train_vectors.nnz)
# # print('Процент заполненности матрицы признаков {:.2f}%'.format(train_vectors.nnz * 100 / (train_vectors.shape[0] * train_vectors.shape[1])))
# # print()
# # print('Количество ненулевых элементов в тестовой выборке', test_vectors.nnz)
# # print('Процент заполненности матрицы признаков {:.2f}%'.format(test_vectors.nnz * 100 / (test_vectors.shape[0] * test_vectors.shape[1])))

# model = nn.Linear(UNIQUE_WORDS_N, UNIQUE_LABELS_N)

# scheduler = lambda optim: \
#     torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=5, factor=0.5, verbose=True)

# experiment_name = 'MAX_DF=0.95_min_token_size=2_tf_lr=1e-2'
# histories[experiment_name], best_models[experiment_name] = train_eval_loop(
#                                             model=model,
#                                             train_dataset=train_dataset,
#                                             val_dataset=test_dataset,
#                                             criterion=F.cross_entropy,
#                                             lr=1e-2,
#                                             epoch_n=200,
#                                             batch_size=32,
#                                             l2_reg_alpha=0,
#                                             lr_scheduler_ctor=scheduler,
#                                             best_acc_type = 'acc',
#                                             test_dataset=test_dataset,
#                                             experiment_name = experiment_name)

# VECTORIZATION_MODE = 'tflidf'
# train_vectors = vectorize_texts(train_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)
# test_vectors = vectorize_texts(test_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)

# # print('Размерность матрицы признаков обучающей выборки', train_vectors.shape)
# # print('Размерность матрицы признаков тестовой выборки', test_vectors.shape)
# # print()
# # print('Количество ненулевых элементов в обучающей выборке', train_vectors.nnz)
# # print('Процент заполненности матрицы признаков {:.2f}%'.format(train_vectors.nnz * 100 / (train_vectors.shape[0] * train_vectors.shape[1])))
# # print()
# # print('Количество ненулевых элементов в тестовой выборке', test_vectors.nnz)
# # print('Процент заполненности матрицы признаков {:.2f}%'.format(test_vectors.nnz * 100 / (test_vectors.shape[0] * test_vectors.shape[1])))

# train_dataset = SparseFeaturesDataset(train_vectors, train_source['target'])
# test_dataset = SparseFeaturesDataset(test_vectors, test_source['target'])

# model = nn.Linear(UNIQUE_WORDS_N, UNIQUE_LABELS_N)

# scheduler = lambda optim: \
#     torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=5, factor=0.5, verbose=True)

# experiment_name = 'MAX_DF=0.95_min_token_size=2_tflidf_lr=1e-2'
# histories[experiment_name], best_models[experiment_name] = train_eval_loop(
#                                             model=model,
#                                             train_dataset=train_dataset,
#                                             val_dataset=test_dataset,
#                                             criterion=F.cross_entropy,
#                                             lr=1e-2,
#                                             epoch_n=200,
#                                             batch_size=32,
#                                             l2_reg_alpha=0,
#                                             lr_scheduler_ctor=scheduler,
#                                             best_acc_type = 'acc',
#                                             test_dataset=test_dataset,
#                                             experiment_name = experiment_name)

In [ ]:
# MAX_DF = 0.60
# MIN_COUNT = 5
# vocabulary, word_doc_freq = build_vocabulary(train_tokenized, max_doc_freq=MAX_DF, min_count=MIN_COUNT)
# UNIQUE_WORDS_N = len(vocabulary)
# print('Количество уникальных токенов', UNIQUE_WORDS_N)
# print(list(vocabulary.items())[:10])

# # word_doc_freq[:20]

# # word_doc_freq[-20:]

# # plt.hist(word_doc_freq, bins=20)
# # plt.title('Распределение относительных частот слов')
# # plt.yscale('log');

# VECTORIZATION_MODE = 'tfidf'
# train_vectors = vectorize_texts(train_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)
# test_vectors = vectorize_texts(test_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)

# print('Размерность матрицы признаков обучающей выборки', train_vectors.shape)
# print('Размерность матрицы признаков тестовой выборки', test_vectors.shape)
# print()
# print('Количество ненулевых элементов в обучающей выборке', train_vectors.nnz)
# print('Процент заполненности матрицы признаков {:.2f}%'.format(train_vectors.nnz * 100 / (train_vectors.shape[0] * train_vectors.shape[1])))
# print()
# print('Количество ненулевых элементов в тестовой выборке', test_vectors.nnz)
# print('Процент заполненности матрицы признаков {:.2f}%'.format(test_vectors.nnz * 100 / (test_vectors.shape[0] * test_vectors.shape[1])))

# # plt.hist(train_vectors.data, bins=20)
# # plt.title('Распределение весов признаков')
# # plt.yscale('log');

# UNIQUE_LABELS_N = len(set(train_source['target']))
# print('Количество уникальных меток', UNIQUE_LABELS_N)
# set(train_source['target'])

# # plt.hist(train_source['target'], bins=np.arange(0, 21))
# # plt.title('Распределение меток в обучающей выборке');

# # plt.hist(test_source['target'], bins=np.arange(0, 21))
# # plt.title('Распределение меток в тестовой выборке');

# train_dataset = SparseFeaturesDataset(train_vectors, train_source['target'])
# test_dataset = SparseFeaturesDataset(test_vectors, test_source['target'])

# model = nn.Linear(UNIQUE_WORDS_N, UNIQUE_LABELS_N)

# scheduler = lambda optim: \
#     torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=5, factor=0.5, verbose=True)

# experiment_name = 'MAX_DF=0.60_min_token_size=2_default'
# histories[experiment_name], best_models[experiment_name] = train_eval_loop(
#                                             model=model,
#                                             train_dataset=train_dataset,
#                                             val_dataset=test_dataset,
#                                             criterion=F.cross_entropy,
#                                             lr=1e-1,
#                                             epoch_n=200,
#                                             batch_size=32,
#                                             l2_reg_alpha=0,
#                                             lr_scheduler_ctor=scheduler,
#                                             best_acc_type = 'acc',
#                                             test_dataset=test_dataset,
#                                             experiment_name = experiment_name)

# VECTORIZATION_MODE = 'tf'
# train_vectors = vectorize_texts(train_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)
# test_vectors = vectorize_texts(test_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)

# # print('Размерность матрицы признаков обучающей выборки', train_vectors.shape)
# # print('Размерность матрицы признаков тестовой выборки', test_vectors.shape)
# # print()
# # print('Количество ненулевых элементов в обучающей выборке', train_vectors.nnz)
# # print('Процент заполненности матрицы признаков {:.2f}%'.format(train_vectors.nnz * 100 / (train_vectors.shape[0] * train_vectors.shape[1])))
# # print()
# # print('Количество ненулевых элементов в тестовой выборке', test_vectors.nnz)
# # print('Процент заполненности матрицы признаков {:.2f}%'.format(test_vectors.nnz * 100 / (test_vectors.shape[0] * test_vectors.shape[1])))

# model = nn.Linear(UNIQUE_WORDS_N, UNIQUE_LABELS_N)

# scheduler = lambda optim: \
#     torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=5, factor=0.5, verbose=True)

# experiment_name = 'MAX_DF=0.60_min_token_size=2_tf_lr=1e-2'
# histories[experiment_name], best_models[experiment_name] = train_eval_loop(
#                                             model=model,
#                                             train_dataset=train_dataset,
#                                             val_dataset=test_dataset,
#                                             criterion=F.cross_entropy,
#                                             lr=1e-2,
#                                             epoch_n=200,
#                                             batch_size=32,
#                                             l2_reg_alpha=0,
#                                             lr_scheduler_ctor=scheduler,
#                                             best_acc_type = 'acc',
#                                             test_dataset=test_dataset,
#                                             experiment_name = experiment_name)

# VECTORIZATION_MODE = 'tflidf'
# train_vectors = vectorize_texts(train_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)
# test_vectors = vectorize_texts(test_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)

# # print('Размерность матрицы признаков обучающей выборки', train_vectors.shape)
# # print('Размерность матрицы признаков тестовой выборки', test_vectors.shape)
# # print()
# # print('Количество ненулевых элементов в обучающей выборке', train_vectors.nnz)
# # print('Процент заполненности матрицы признаков {:.2f}%'.format(train_vectors.nnz * 100 / (train_vectors.shape[0] * train_vectors.shape[1])))
# # print()
# # print('Количество ненулевых элементов в тестовой выборке', test_vectors.nnz)
# # print('Процент заполненности матрицы признаков {:.2f}%'.format(test_vectors.nnz * 100 / (test_vectors.shape[0] * test_vectors.shape[1])))

# train_dataset = SparseFeaturesDataset(train_vectors, train_source['target'])
# test_dataset = SparseFeaturesDataset(test_vectors, test_source['target'])

# model = nn.Linear(UNIQUE_WORDS_N, UNIQUE_LABELS_N)

# scheduler = lambda optim: \
#     torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=5, factor=0.5, verbose=True)

# experiment_name = 'MAX_DF=0.60_min_token_size=2_tflidf_lr=1e-2'
# histories[experiment_name], best_models[experiment_name] = train_eval_loop(
#                                             model=model,
#                                             train_dataset=train_dataset,
#                                             val_dataset=test_dataset,
#                                             criterion=F.cross_entropy,
#                                             lr=1e-2,
#                                             epoch_n=200,
#                                             batch_size=32,
#                                             l2_reg_alpha=0,
#                                             lr_scheduler_ctor=scheduler,
#                                             best_acc_type = 'acc',
#                                             test_dataset=test_dataset,
#                                             experiment_name = experiment_name)

In [ ]:
# MAX_DF = 0.30
# MIN_COUNT = 5
# vocabulary, word_doc_freq = build_vocabulary(train_tokenized, max_doc_freq=MAX_DF, min_count=MIN_COUNT)
# UNIQUE_WORDS_N = len(vocabulary)
# print('Количество уникальных токенов', UNIQUE_WORDS_N)
# print(list(vocabulary.items())[:10])

# # word_doc_freq[:20]

# # word_doc_freq[-20:]

# # plt.hist(word_doc_freq, bins=20)
# # plt.title('Распределение относительных частот слов')
# # plt.yscale('log');

# VECTORIZATION_MODE = 'tfidf'
# train_vectors = vectorize_texts(train_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)
# test_vectors = vectorize_texts(test_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)

# print('Размерность матрицы признаков обучающей выборки', train_vectors.shape)
# print('Размерность матрицы признаков тестовой выборки', test_vectors.shape)
# print()
# print('Количество ненулевых элементов в обучающей выборке', train_vectors.nnz)
# print('Процент заполненности матрицы признаков {:.2f}%'.format(train_vectors.nnz * 100 / (train_vectors.shape[0] * train_vectors.shape[1])))
# print()
# print('Количество ненулевых элементов в тестовой выборке', test_vectors.nnz)
# print('Процент заполненности матрицы признаков {:.2f}%'.format(test_vectors.nnz * 100 / (test_vectors.shape[0] * test_vectors.shape[1])))

# # plt.hist(train_vectors.data, bins=20)
# # plt.title('Распределение весов признаков')
# # plt.yscale('log');

# UNIQUE_LABELS_N = len(set(train_source['target']))
# print('Количество уникальных меток', UNIQUE_LABELS_N)
# set(train_source['target'])

# # plt.hist(train_source['target'], bins=np.arange(0, 21))
# # plt.title('Распределение меток в обучающей выборке');

# # plt.hist(test_source['target'], bins=np.arange(0, 21))
# # plt.title('Распределение меток в тестовой выборке');

# train_dataset = SparseFeaturesDataset(train_vectors, train_source['target'])
# test_dataset = SparseFeaturesDataset(test_vectors, test_source['target'])

# model = nn.Linear(UNIQUE_WORDS_N, UNIQUE_LABELS_N)

# scheduler = lambda optim: \
#     torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=5, factor=0.5, verbose=True)

# experiment_name = 'MAX_DF=0.30_min_token_size=2_default'
# histories[experiment_name], best_models[experiment_name] = train_eval_loop(
#                                             model=model,
#                                             train_dataset=train_dataset,
#                                             val_dataset=test_dataset,
#                                             criterion=F.cross_entropy,
#                                             lr=1e-1,
#                                             epoch_n=200,
#                                             batch_size=32,
#                                             l2_reg_alpha=0,
#                                             lr_scheduler_ctor=scheduler,
#                                             best_acc_type = 'acc',
#                                             test_dataset=test_dataset,
#                                             experiment_name = experiment_name)

# VECTORIZATION_MODE = 'tf'
# train_vectors = vectorize_texts(train_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)
# test_vectors = vectorize_texts(test_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)

# # print('Размерность матрицы признаков обучающей выборки', train_vectors.shape)
# # print('Размерность матрицы признаков тестовой выборки', test_vectors.shape)
# # print()
# # print('Количество ненулевых элементов в обучающей выборке', train_vectors.nnz)
# # print('Процент заполненности матрицы признаков {:.2f}%'.format(train_vectors.nnz * 100 / (train_vectors.shape[0] * train_vectors.shape[1])))
# # print()
# # print('Количество ненулевых элементов в тестовой выборке', test_vectors.nnz)
# # print('Процент заполненности матрицы признаков {:.2f}%'.format(test_vectors.nnz * 100 / (test_vectors.shape[0] * test_vectors.shape[1])))

# model = nn.Linear(UNIQUE_WORDS_N, UNIQUE_LABELS_N)

# scheduler = lambda optim: \
#     torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=5, factor=0.5, verbose=True)

# experiment_name = 'MAX_DF=0.30_min_token_size=2_tf_lr=1e-2'
# histories[experiment_name], best_models[experiment_name] = train_eval_loop(
#                                             model=model,
#                                             train_dataset=train_dataset,
#                                             val_dataset=test_dataset,
#                                             criterion=F.cross_entropy,
#                                             lr=1e-2,
#                                             epoch_n=200,
#                                             batch_size=32,
#                                             l2_reg_alpha=0,
#                                             lr_scheduler_ctor=scheduler,
#                                             best_acc_type = 'acc',
#                                             test_dataset=test_dataset,
#                                             experiment_name = experiment_name)

# VECTORIZATION_MODE = 'tflidf'
# train_vectors = vectorize_texts(train_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)
# test_vectors = vectorize_texts(test_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)

# # print('Размерность матрицы признаков обучающей выборки', train_vectors.shape)
# # print('Размерность матрицы признаков тестовой выборки', test_vectors.shape)
# # print()
# # print('Количество ненулевых элементов в обучающей выборке', train_vectors.nnz)
# # print('Процент заполненности матрицы признаков {:.2f}%'.format(train_vectors.nnz * 100 / (train_vectors.shape[0] * train_vectors.shape[1])))
# # print()
# # print('Количество ненулевых элементов в тестовой выборке', test_vectors.nnz)
# # print('Процент заполненности матрицы признаков {:.2f}%'.format(test_vectors.nnz * 100 / (test_vectors.shape[0] * test_vectors.shape[1])))

# train_dataset = SparseFeaturesDataset(train_vectors, train_source['target'])
# test_dataset = SparseFeaturesDataset(test_vectors, test_source['target'])

# model = nn.Linear(UNIQUE_WORDS_N, UNIQUE_LABELS_N)

# scheduler = lambda optim: \
#     torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=5, factor=0.5, verbose=True)

# experiment_name = 'MAX_DF=0.30_min_token_size=2_tflidf_lr=1e-2'
# histories[experiment_name], best_models[experiment_name] = train_eval_loop(
#                                             model=model,
#                                             train_dataset=train_dataset,
#                                             val_dataset=test_dataset,
#                                             criterion=F.cross_entropy,
#                                             lr=1e-2,
#                                             epoch_n=200,
#                                             batch_size=32,
#                                             l2_reg_alpha=0,
#                                             lr_scheduler_ctor=scheduler,
#                                             best_acc_type = 'acc',
#                                             test_dataset=test_dataset,
#                                             experiment_name = experiment_name)

In [ ]:
# MAX_DF = 0.80
# MIN_COUNT = 2
# vocabulary, word_doc_freq = build_vocabulary(train_tokenized, max_doc_freq=MAX_DF, min_count=MIN_COUNT)
# UNIQUE_WORDS_N = len(vocabulary)
# print('Количество уникальных токенов', UNIQUE_WORDS_N)
# print(list(vocabulary.items())[:10])

# # word_doc_freq[:20]

# # word_doc_freq[-20:]

# # plt.hist(word_doc_freq, bins=20)
# # plt.title('Распределение относительных частот слов')
# # plt.yscale('log');

# VECTORIZATION_MODE = 'tfidf'
# train_vectors = vectorize_texts(train_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)
# test_vectors = vectorize_texts(test_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)

# print('Размерность матрицы признаков обучающей выборки', train_vectors.shape)
# print('Размерность матрицы признаков тестовой выборки', test_vectors.shape)
# print()
# print('Количество ненулевых элементов в обучающей выборке', train_vectors.nnz)
# print('Процент заполненности матрицы признаков {:.2f}%'.format(train_vectors.nnz * 100 / (train_vectors.shape[0] * train_vectors.shape[1])))
# print()
# print('Количество ненулевых элементов в тестовой выборке', test_vectors.nnz)
# print('Процент заполненности матрицы признаков {:.2f}%'.format(test_vectors.nnz * 100 / (test_vectors.shape[0] * test_vectors.shape[1])))

# # plt.hist(train_vectors.data, bins=20)
# # plt.title('Распределение весов признаков')
# # plt.yscale('log');

# UNIQUE_LABELS_N = len(set(train_source['target']))
# print('Количество уникальных меток', UNIQUE_LABELS_N)
# set(train_source['target'])

# # plt.hist(train_source['target'], bins=np.arange(0, 21))
# # plt.title('Распределение меток в обучающей выборке');

# # plt.hist(test_source['target'], bins=np.arange(0, 21))
# # plt.title('Распределение меток в тестовой выборке');

# train_dataset = SparseFeaturesDataset(train_vectors, train_source['target'])
# test_dataset = SparseFeaturesDataset(test_vectors, test_source['target'])

# model = nn.Linear(UNIQUE_WORDS_N, UNIQUE_LABELS_N)

# scheduler = lambda optim: \
#     torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=5, factor=0.5, verbose=True)

# experiment_name = 'MIN_COUNT=2_min_token_size=2_default'
# histories[experiment_name], best_models[experiment_name] = train_eval_loop(
#                                             model=model,
#                                             train_dataset=train_dataset,
#                                             val_dataset=test_dataset,
#                                             criterion=F.cross_entropy,
#                                             lr=1e-1,
#                                             epoch_n=200,
#                                             batch_size=32,
#                                             l2_reg_alpha=0,
#                                             lr_scheduler_ctor=scheduler,
#                                             best_acc_type = 'acc',
#                                             test_dataset=test_dataset,
#                                             experiment_name = experiment_name)

# VECTORIZATION_MODE = 'tf'
# train_vectors = vectorize_texts(train_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)
# test_vectors = vectorize_texts(test_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)

# # print('Размерность матрицы признаков обучающей выборки', train_vectors.shape)
# # print('Размерность матрицы признаков тестовой выборки', test_vectors.shape)
# # print()
# # print('Количество ненулевых элементов в обучающей выборке', train_vectors.nnz)
# # print('Процент заполненности матрицы признаков {:.2f}%'.format(train_vectors.nnz * 100 / (train_vectors.shape[0] * train_vectors.shape[1])))
# # print()
# # print('Количество ненулевых элементов в тестовой выборке', test_vectors.nnz)
# # print('Процент заполненности матрицы признаков {:.2f}%'.format(test_vectors.nnz * 100 / (test_vectors.shape[0] * test_vectors.shape[1])))

# model = nn.Linear(UNIQUE_WORDS_N, UNIQUE_LABELS_N)

# scheduler = lambda optim: \
#     torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=5, factor=0.5, verbose=True)

# experiment_name = 'MIN_COUNT=2_min_token_size=2_tf_lr=1e-2'
# histories[experiment_name], best_models[experiment_name] = train_eval_loop(
#                                             model=model,
#                                             train_dataset=train_dataset,
#                                             val_dataset=test_dataset,
#                                             criterion=F.cross_entropy,
#                                             lr=1e-2,
#                                             epoch_n=200,
#                                             batch_size=32,
#                                             l2_reg_alpha=0,
#                                             lr_scheduler_ctor=scheduler,
#                                             best_acc_type = 'acc',
#                                             test_dataset=test_dataset,
#                                             experiment_name = experiment_name)

# VECTORIZATION_MODE = 'tflidf'
# train_vectors = vectorize_texts(train_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)
# test_vectors = vectorize_texts(test_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)

# # print('Размерность матрицы признаков обучающей выборки', train_vectors.shape)
# # print('Размерность матрицы признаков тестовой выборки', test_vectors.shape)
# # print()
# # print('Количество ненулевых элементов в обучающей выборке', train_vectors.nnz)
# # print('Процент заполненности матрицы признаков {:.2f}%'.format(train_vectors.nnz * 100 / (train_vectors.shape[0] * train_vectors.shape[1])))
# # print()
# # print('Количество ненулевых элементов в тестовой выборке', test_vectors.nnz)
# # print('Процент заполненности матрицы признаков {:.2f}%'.format(test_vectors.nnz * 100 / (test_vectors.shape[0] * test_vectors.shape[1])))

# train_dataset = SparseFeaturesDataset(train_vectors, train_source['target'])
# test_dataset = SparseFeaturesDataset(test_vectors, test_source['target'])

# model = nn.Linear(UNIQUE_WORDS_N, UNIQUE_LABELS_N)

# scheduler = lambda optim: \
#     torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=5, factor=0.5, verbose=True)

# experiment_name = 'MIN_COUNT=2_min_token_size=2_tflidf_lr=1e-2'
# histories[experiment_name], best_models[experiment_name] = train_eval_loop(
#                                             model=model,
#                                             train_dataset=train_dataset,
#                                             val_dataset=test_dataset,
#                                             criterion=F.cross_entropy,
#                                             lr=1e-2,
#                                             epoch_n=200,
#                                             batch_size=32,
#                                             l2_reg_alpha=0,
#                                             lr_scheduler_ctor=scheduler,
#                                             best_acc_type = 'acc',
#                                             test_dataset=test_dataset,
#                                             experiment_name = experiment_name)

In [ ]:
# MAX_DF = 0.80
# MIN_COUNT = 20
# vocabulary, word_doc_freq = build_vocabulary(train_tokenized, max_doc_freq=MAX_DF, min_count=MIN_COUNT)
# UNIQUE_WORDS_N = len(vocabulary)
# print('Количество уникальных токенов', UNIQUE_WORDS_N)
# print(list(vocabulary.items())[:10])

# # word_doc_freq[:20]

# # word_doc_freq[-20:]

# # plt.hist(word_doc_freq, bins=20)
# # plt.title('Распределение относительных частот слов')
# # plt.yscale('log');

# VECTORIZATION_MODE = 'tfidf'
# train_vectors = vectorize_texts(train_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)
# test_vectors = vectorize_texts(test_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)

# print('Размерность матрицы признаков обучающей выборки', train_vectors.shape)
# print('Размерность матрицы признаков тестовой выборки', test_vectors.shape)
# print()
# print('Количество ненулевых элементов в обучающей выборке', train_vectors.nnz)
# print('Процент заполненности матрицы признаков {:.2f}%'.format(train_vectors.nnz * 100 / (train_vectors.shape[0] * train_vectors.shape[1])))
# print()
# print('Количество ненулевых элементов в тестовой выборке', test_vectors.nnz)
# print('Процент заполненности матрицы признаков {:.2f}%'.format(test_vectors.nnz * 100 / (test_vectors.shape[0] * test_vectors.shape[1])))

# # plt.hist(train_vectors.data, bins=20)
# # plt.title('Распределение весов признаков')
# # plt.yscale('log');

# UNIQUE_LABELS_N = len(set(train_source['target']))
# print('Количество уникальных меток', UNIQUE_LABELS_N)
# set(train_source['target'])

# # plt.hist(train_source['target'], bins=np.arange(0, 21))
# # plt.title('Распределение меток в обучающей выборке');

# # plt.hist(test_source['target'], bins=np.arange(0, 21))
# # plt.title('Распределение меток в тестовой выборке');

# train_dataset = SparseFeaturesDataset(train_vectors, train_source['target'])
# test_dataset = SparseFeaturesDataset(test_vectors, test_source['target'])

# model = nn.Linear(UNIQUE_WORDS_N, UNIQUE_LABELS_N)

# scheduler = lambda optim: \
#     torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=5, factor=0.5, verbose=True)

# experiment_name = 'MIN_COUNT=20_min_token_size=2_default'
# histories[experiment_name], best_models[experiment_name] = train_eval_loop(
#                                             model=model,
#                                             train_dataset=train_dataset,
#                                             val_dataset=test_dataset,
#                                             criterion=F.cross_entropy,
#                                             lr=1e-1,
#                                             epoch_n=200,
#                                             batch_size=32,
#                                             l2_reg_alpha=0,
#                                             lr_scheduler_ctor=scheduler,
#                                             best_acc_type = 'acc',
#                                             test_dataset=test_dataset,
#                                             experiment_name = experiment_name)

# VECTORIZATION_MODE = 'tf'
# train_vectors = vectorize_texts(train_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)
# test_vectors = vectorize_texts(test_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)

# # print('Размерность матрицы признаков обучающей выборки', train_vectors.shape)
# # print('Размерность матрицы признаков тестовой выборки', test_vectors.shape)
# # print()
# # print('Количество ненулевых элементов в обучающей выборке', train_vectors.nnz)
# # print('Процент заполненности матрицы признаков {:.2f}%'.format(train_vectors.nnz * 100 / (train_vectors.shape[0] * train_vectors.shape[1])))
# # print()
# # print('Количество ненулевых элементов в тестовой выборке', test_vectors.nnz)
# # print('Процент заполненности матрицы признаков {:.2f}%'.format(test_vectors.nnz * 100 / (test_vectors.shape[0] * test_vectors.shape[1])))

# model = nn.Linear(UNIQUE_WORDS_N, UNIQUE_LABELS_N)

# scheduler = lambda optim: \
#     torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=5, factor=0.5, verbose=True)

# experiment_name = 'MIN_COUNT=20_tf_min_token_size=2_lr=1e-2'
# histories[experiment_name], best_models[experiment_name] = train_eval_loop(
#                                             model=model,
#                                             train_dataset=train_dataset,
#                                             val_dataset=test_dataset,
#                                             criterion=F.cross_entropy,
#                                             lr=1e-2,
#                                             epoch_n=200,
#                                             batch_size=32,
#                                             l2_reg_alpha=0,
#                                             lr_scheduler_ctor=scheduler,
#                                             best_acc_type = 'acc',
#                                             test_dataset=test_dataset,
#                                             experiment_name = experiment_name)

# VECTORIZATION_MODE = 'tflidf'
# train_vectors = vectorize_texts(train_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)
# test_vectors = vectorize_texts(test_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)

# # print('Размерность матрицы признаков обучающей выборки', train_vectors.shape)
# # print('Размерность матрицы признаков тестовой выборки', test_vectors.shape)
# # print()
# # print('Количество ненулевых элементов в обучающей выборке', train_vectors.nnz)
# # print('Процент заполненности матрицы признаков {:.2f}%'.format(train_vectors.nnz * 100 / (train_vectors.shape[0] * train_vectors.shape[1])))
# # print()
# # print('Количество ненулевых элементов в тестовой выборке', test_vectors.nnz)
# # print('Процент заполненности матрицы признаков {:.2f}%'.format(test_vectors.nnz * 100 / (test_vectors.shape[0] * test_vectors.shape[1])))

# train_dataset = SparseFeaturesDataset(train_vectors, train_source['target'])
# test_dataset = SparseFeaturesDataset(test_vectors, test_source['target'])

# model = nn.Linear(UNIQUE_WORDS_N, UNIQUE_LABELS_N)

# scheduler = lambda optim: \
#     torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=5, factor=0.5, verbose=True)

# experiment_name = 'MIN_COUNT=20_tflidf_min_token_size=2_lr=1e-2'
# histories[experiment_name], best_models[experiment_name] = train_eval_loop(
#                                             model=model,
#                                             train_dataset=train_dataset,
#                                             val_dataset=test_dataset,
#                                             criterion=F.cross_entropy,
#                                             lr=1e-2,
#                                             epoch_n=200,
#                                             batch_size=32,
#                                             l2_reg_alpha=0,
#                                             lr_scheduler_ctor=scheduler,
#                                             best_acc_type = 'acc',
#                                             test_dataset=test_dataset,
#                                             experiment_name = experiment_name)

In [ ]:
# MAX_DF = 0.80
# MIN_COUNT = 80
# vocabulary, word_doc_freq = build_vocabulary(train_tokenized, max_doc_freq=MAX_DF, min_count=MIN_COUNT)
# UNIQUE_WORDS_N = len(vocabulary)
# print('Количество уникальных токенов', UNIQUE_WORDS_N)
# print(list(vocabulary.items())[:10])

# # word_doc_freq[:20]

# # word_doc_freq[-20:]

# # plt.hist(word_doc_freq, bins=20)
# # plt.title('Распределение относительных частот слов')
# # plt.yscale('log');

# VECTORIZATION_MODE = 'tfidf'
# train_vectors = vectorize_texts(train_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)
# test_vectors = vectorize_texts(test_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)

# print('Размерность матрицы признаков обучающей выборки', train_vectors.shape)
# print('Размерность матрицы признаков тестовой выборки', test_vectors.shape)
# print()
# print('Количество ненулевых элементов в обучающей выборке', train_vectors.nnz)
# print('Процент заполненности матрицы признаков {:.2f}%'.format(train_vectors.nnz * 100 / (train_vectors.shape[0] * train_vectors.shape[1])))
# print()
# print('Количество ненулевых элементов в тестовой выборке', test_vectors.nnz)
# print('Процент заполненности матрицы признаков {:.2f}%'.format(test_vectors.nnz * 100 / (test_vectors.shape[0] * test_vectors.shape[1])))

# # plt.hist(train_vectors.data, bins=20)
# # plt.title('Распределение весов признаков')
# # plt.yscale('log');

# UNIQUE_LABELS_N = len(set(train_source['target']))
# print('Количество уникальных меток', UNIQUE_LABELS_N)
# set(train_source['target'])

# # plt.hist(train_source['target'], bins=np.arange(0, 21))
# # plt.title('Распределение меток в обучающей выборке');

# # plt.hist(test_source['target'], bins=np.arange(0, 21))
# # plt.title('Распределение меток в тестовой выборке');

# train_dataset = SparseFeaturesDataset(train_vectors, train_source['target'])
# test_dataset = SparseFeaturesDataset(test_vectors, test_source['target'])

# model = nn.Linear(UNIQUE_WORDS_N, UNIQUE_LABELS_N)

# scheduler = lambda optim: \
#     torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=5, factor=0.5, verbose=True)

# experiment_name = 'MIN_COUNT=30_min_token_size=2_default'
# histories[experiment_name], best_models[experiment_name] = train_eval_loop(
#                                             model=model,
#                                             train_dataset=train_dataset,
#                                             val_dataset=test_dataset,
#                                             criterion=F.cross_entropy,
#                                             lr=1e-1,
#                                             epoch_n=200,
#                                             batch_size=32,
#                                             l2_reg_alpha=0,
#                                             lr_scheduler_ctor=scheduler,
#                                             best_acc_type = 'acc',
#                                             test_dataset=test_dataset,
#                                             experiment_name = experiment_name)

# VECTORIZATION_MODE = 'tf'
# train_vectors = vectorize_texts(train_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)
# test_vectors = vectorize_texts(test_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)

# # print('Размерность матрицы признаков обучающей выборки', train_vectors.shape)
# # print('Размерность матрицы признаков тестовой выборки', test_vectors.shape)
# # print()
# # print('Количество ненулевых элементов в обучающей выборке', train_vectors.nnz)
# # print('Процент заполненности матрицы признаков {:.2f}%'.format(train_vectors.nnz * 100 / (train_vectors.shape[0] * train_vectors.shape[1])))
# # print()
# # print('Количество ненулевых элементов в тестовой выборке', test_vectors.nnz)
# # print('Процент заполненности матрицы признаков {:.2f}%'.format(test_vectors.nnz * 100 / (test_vectors.shape[0] * test_vectors.shape[1])))

# model = nn.Linear(UNIQUE_WORDS_N, UNIQUE_LABELS_N)

# scheduler = lambda optim: \
#     torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=5, factor=0.5, verbose=True)

# experiment_name = 'MIN_COUNT=80_min_token_size=2_tf_lr=1e-2'
# histories[experiment_name], best_models[experiment_name] = train_eval_loop(
#                                             model=model,
#                                             train_dataset=train_dataset,
#                                             val_dataset=test_dataset,
#                                             criterion=F.cross_entropy,
#                                             lr=1e-2,
#                                             epoch_n=200,
#                                             batch_size=32,
#                                             l2_reg_alpha=0,
#                                             lr_scheduler_ctor=scheduler,
#                                             best_acc_type = 'acc',
#                                             test_dataset=test_dataset,
#                                             experiment_name = experiment_name)

# VECTORIZATION_MODE = 'tflidf'
# train_vectors = vectorize_texts(train_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)
# test_vectors = vectorize_texts(test_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)

# # print('Размерность матрицы признаков обучающей выборки', train_vectors.shape)
# # print('Размерность матрицы признаков тестовой выборки', test_vectors.shape)
# # print()
# # print('Количество ненулевых элементов в обучающей выборке', train_vectors.nnz)
# # print('Процент заполненности матрицы признаков {:.2f}%'.format(train_vectors.nnz * 100 / (train_vectors.shape[0] * train_vectors.shape[1])))
# # print()
# # print('Количество ненулевых элементов в тестовой выборке', test_vectors.nnz)
# # print('Процент заполненности матрицы признаков {:.2f}%'.format(test_vectors.nnz * 100 / (test_vectors.shape[0] * test_vectors.shape[1])))

# train_dataset = SparseFeaturesDataset(train_vectors, train_source['target'])
# test_dataset = SparseFeaturesDataset(test_vectors, test_source['target'])

# model = nn.Linear(UNIQUE_WORDS_N, UNIQUE_LABELS_N)

# scheduler = lambda optim: \
#     torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=5, factor=0.5, verbose=True)

# experiment_name = 'MIN_COUNT=80_min_token_size=2_tflidf_lr=1e-2'
# histories[experiment_name], best_models[experiment_name] = train_eval_loop(
#                                             model=model,
#                                             train_dataset=train_dataset,
#                                             val_dataset=test_dataset,
#                                             criterion=F.cross_entropy,
#                                             lr=1e-2,
#                                             epoch_n=200,
#                                             batch_size=32,
#                                             l2_reg_alpha=0,
#                                             lr_scheduler_ctor=scheduler,
#                                             best_acc_type = 'acc',
#                                             test_dataset=test_dataset,
#                                             experiment_name = experiment_name)

In [ ]:
# MAX_DF = 0.98
# MIN_COUNT = 2
# vocabulary, word_doc_freq = build_vocabulary(train_tokenized, max_doc_freq=MAX_DF, min_count=MIN_COUNT)
# UNIQUE_WORDS_N = len(vocabulary)
# print('Количество уникальных токенов', UNIQUE_WORDS_N)
# print(list(vocabulary.items())[:10])

# # word_doc_freq[:20]

# # word_doc_freq[-20:]

# # plt.hist(word_doc_freq, bins=20)
# # plt.title('Распределение относительных частот слов')
# # plt.yscale('log');

# VECTORIZATION_MODE = 'tfidf'
# train_vectors = vectorize_texts(train_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)
# test_vectors = vectorize_texts(test_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)

# print('Размерность матрицы признаков обучающей выборки', train_vectors.shape)
# print('Размерность матрицы признаков тестовой выборки', test_vectors.shape)
# print()
# print('Количество ненулевых элементов в обучающей выборке', train_vectors.nnz)
# print('Процент заполненности матрицы признаков {:.2f}%'.format(train_vectors.nnz * 100 / (train_vectors.shape[0] * train_vectors.shape[1])))
# print()
# print('Количество ненулевых элементов в тестовой выборке', test_vectors.nnz)
# print('Процент заполненности матрицы признаков {:.2f}%'.format(test_vectors.nnz * 100 / (test_vectors.shape[0] * test_vectors.shape[1])))

# # plt.hist(train_vectors.data, bins=20)
# # plt.title('Распределение весов признаков')
# # plt.yscale('log');

# UNIQUE_LABELS_N = len(set(train_source['target']))
# print('Количество уникальных меток', UNIQUE_LABELS_N)
# set(train_source['target'])

# # plt.hist(train_source['target'], bins=np.arange(0, 21))
# # plt.title('Распределение меток в обучающей выборке');

# # plt.hist(test_source['target'], bins=np.arange(0, 21))
# # plt.title('Распределение меток в тестовой выборке');

# train_dataset = SparseFeaturesDataset(train_vectors, train_source['target'])
# test_dataset = SparseFeaturesDataset(test_vectors, test_source['target'])

# model = nn.Linear(UNIQUE_WORDS_N, UNIQUE_LABELS_N)

# scheduler = lambda optim: \
#     torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=5, factor=0.5, verbose=True)

# experiment_name = 'MAX_DF=0.98_MIN_COUNT=2_min_token_size=2_default'
# histories[experiment_name], best_models[experiment_name] = train_eval_loop(
#                                             model=model,
#                                             train_dataset=train_dataset,
#                                             val_dataset=test_dataset,
#                                             criterion=F.cross_entropy,
#                                             lr=1e-1,
#                                             epoch_n=200,
#                                             batch_size=32,
#                                             l2_reg_alpha=0,
#                                             lr_scheduler_ctor=scheduler,
#                                             best_acc_type = 'acc',
#                                             test_dataset=test_dataset,
#                                             experiment_name = experiment_name)

# VECTORIZATION_MODE = 'tf'
# train_vectors = vectorize_texts(train_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)
# test_vectors = vectorize_texts(test_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)

# # print('Размерность матрицы признаков обучающей выборки', train_vectors.shape)
# # print('Размерность матрицы признаков тестовой выборки', test_vectors.shape)
# # print()
# # print('Количество ненулевых элементов в обучающей выборке', train_vectors.nnz)
# # print('Процент заполненности матрицы признаков {:.2f}%'.format(train_vectors.nnz * 100 / (train_vectors.shape[0] * train_vectors.shape[1])))
# # print()
# # print('Количество ненулевых элементов в тестовой выборке', test_vectors.nnz)
# # print('Процент заполненности матрицы признаков {:.2f}%'.format(test_vectors.nnz * 100 / (test_vectors.shape[0] * test_vectors.shape[1])))

# model = nn.Linear(UNIQUE_WORDS_N, UNIQUE_LABELS_N)

# scheduler = lambda optim: \
#     torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=5, factor=0.5, verbose=True)

# experiment_name = 'MAX_DF=0.98_MIN_COUNT=2_min_token_size=2_tf_lr=1e-2'
# histories[experiment_name], best_models[experiment_name] = train_eval_loop(
#                                             model=model,
#                                             train_dataset=train_dataset,
#                                             val_dataset=test_dataset,
#                                             criterion=F.cross_entropy,
#                                             lr=1e-2,
#                                             epoch_n=200,
#                                             batch_size=32,
#                                             l2_reg_alpha=0,
#                                             lr_scheduler_ctor=scheduler,
#                                             best_acc_type = 'acc',
#                                             test_dataset=test_dataset,
#                                             experiment_name = experiment_name)

# VECTORIZATION_MODE = 'tflidf'
# train_vectors = vectorize_texts(train_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)
# test_vectors = vectorize_texts(test_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)

# # print('Размерность матрицы признаков обучающей выборки', train_vectors.shape)
# # print('Размерность матрицы признаков тестовой выборки', test_vectors.shape)
# # print()
# # print('Количество ненулевых элементов в обучающей выборке', train_vectors.nnz)
# # print('Процент заполненности матрицы признаков {:.2f}%'.format(train_vectors.nnz * 100 / (train_vectors.shape[0] * train_vectors.shape[1])))
# # print()
# # print('Количество ненулевых элементов в тестовой выборке', test_vectors.nnz)
# # print('Процент заполненности матрицы признаков {:.2f}%'.format(test_vectors.nnz * 100 / (test_vectors.shape[0] * test_vectors.shape[1])))

# train_dataset = SparseFeaturesDataset(train_vectors, train_source['target'])
# test_dataset = SparseFeaturesDataset(test_vectors, test_source['target'])

# model = nn.Linear(UNIQUE_WORDS_N, UNIQUE_LABELS_N)

# scheduler = lambda optim: \
#     torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=5, factor=0.5, verbose=True)

# experiment_name = 'MAX_DF=0.98_MIN_COUNT=2_min_token_size=2_tflidf_lr=1e-2'
# histories[experiment_name], best_models[experiment_name] = train_eval_loop(
#                                             model=model,
#                                             train_dataset=train_dataset,
#                                             val_dataset=test_dataset,
#                                             criterion=F.cross_entropy,
#                                             lr=1e-2,
#                                             epoch_n=200,
#                                             batch_size=32,
#                                             l2_reg_alpha=0,
#                                             lr_scheduler_ctor=scheduler,
#                                             best_acc_type = 'acc',
#                                             test_dataset=test_dataset,
#                                             experiment_name = experiment_name)

In [ ]:
# MAX_DF = 0.60
# MIN_COUNT = 80
# vocabulary, word_doc_freq = build_vocabulary(train_tokenized, max_doc_freq=MAX_DF, min_count=MIN_COUNT)
# UNIQUE_WORDS_N = len(vocabulary)
# print('Количество уникальных токенов', UNIQUE_WORDS_N)
# print(list(vocabulary.items())[:10])

# # word_doc_freq[:20]

# # word_doc_freq[-20:]

# # plt.hist(word_doc_freq, bins=20)
# # plt.title('Распределение относительных частот слов')
# # plt.yscale('log');

# VECTORIZATION_MODE = 'tfidf'
# train_vectors = vectorize_texts(train_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)
# test_vectors = vectorize_texts(test_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)

# print('Размерность матрицы признаков обучающей выборки', train_vectors.shape)
# print('Размерность матрицы признаков тестовой выборки', test_vectors.shape)
# print()
# print('Количество ненулевых элементов в обучающей выборке', train_vectors.nnz)
# print('Процент заполненности матрицы признаков {:.2f}%'.format(train_vectors.nnz * 100 / (train_vectors.shape[0] * train_vectors.shape[1])))
# print()
# print('Количество ненулевых элементов в тестовой выборке', test_vectors.nnz)
# print('Процент заполненности матрицы признаков {:.2f}%'.format(test_vectors.nnz * 100 / (test_vectors.shape[0] * test_vectors.shape[1])))

# # plt.hist(train_vectors.data, bins=20)
# # plt.title('Распределение весов признаков')
# # plt.yscale('log');

# UNIQUE_LABELS_N = len(set(train_source['target']))
# print('Количество уникальных меток', UNIQUE_LABELS_N)
# set(train_source['target'])

# # plt.hist(train_source['target'], bins=np.arange(0, 21))
# # plt.title('Распределение меток в обучающей выборке');

# # plt.hist(test_source['target'], bins=np.arange(0, 21))
# # plt.title('Распределение меток в тестовой выборке');

# train_dataset = SparseFeaturesDataset(train_vectors, train_source['target'])
# test_dataset = SparseFeaturesDataset(test_vectors, test_source['target'])

# model = nn.Linear(UNIQUE_WORDS_N, UNIQUE_LABELS_N)

# scheduler = lambda optim: \
#     torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=5, factor=0.5, verbose=True)

# experiment_name = 'MAX_DF=0.60_MIN_COUNT=80_min_token_size=2_default'
# histories[experiment_name], best_models[experiment_name] = train_eval_loop(
#                                             model=model,
#                                             train_dataset=train_dataset,
#                                             val_dataset=test_dataset,
#                                             criterion=F.cross_entropy,
#                                             lr=1e-1,
#                                             epoch_n=200,
#                                             batch_size=32,
#                                             l2_reg_alpha=0,
#                                             lr_scheduler_ctor=scheduler,
#                                             best_acc_type = 'acc',
#                                             test_dataset=test_dataset,
#                                             experiment_name = experiment_name)

# VECTORIZATION_MODE = 'tf'
# train_vectors = vectorize_texts(train_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)
# test_vectors = vectorize_texts(test_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)

# # print('Размерность матрицы признаков обучающей выборки', train_vectors.shape)
# # print('Размерность матрицы признаков тестовой выборки', test_vectors.shape)
# # print()
# # print('Количество ненулевых элементов в обучающей выборке', train_vectors.nnz)
# # print('Процент заполненности матрицы признаков {:.2f}%'.format(train_vectors.nnz * 100 / (train_vectors.shape[0] * train_vectors.shape[1])))
# # print()
# # print('Количество ненулевых элементов в тестовой выборке', test_vectors.nnz)
# # print('Процент заполненности матрицы признаков {:.2f}%'.format(test_vectors.nnz * 100 / (test_vectors.shape[0] * test_vectors.shape[1])))

# model = nn.Linear(UNIQUE_WORDS_N, UNIQUE_LABELS_N)

# scheduler = lambda optim: \
#     torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=5, factor=0.5, verbose=True)

# experiment_name = 'MAX_DF=0.60_MIN_COUNT=80_min_token_size=2_tf_lr=1e-2'
# histories[experiment_name], best_models[experiment_name] = train_eval_loop(
#                                             model=model,
#                                             train_dataset=train_dataset,
#                                             val_dataset=test_dataset,
#                                             criterion=F.cross_entropy,
#                                             lr=1e-2,
#                                             epoch_n=200,
#                                             batch_size=32,
#                                             l2_reg_alpha=0,
#                                             lr_scheduler_ctor=scheduler,
#                                             best_acc_type = 'acc',
#                                             test_dataset=test_dataset,
#                                             experiment_name = experiment_name)

# VECTORIZATION_MODE = 'tflidf'
# train_vectors = vectorize_texts(train_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)
# test_vectors = vectorize_texts(test_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)

# # print('Размерность матрицы признаков обучающей выборки', train_vectors.shape)
# # print('Размерность матрицы признаков тестовой выборки', test_vectors.shape)
# # print()
# # print('Количество ненулевых элементов в обучающей выборке', train_vectors.nnz)
# # print('Процент заполненности матрицы признаков {:.2f}%'.format(train_vectors.nnz * 100 / (train_vectors.shape[0] * train_vectors.shape[1])))
# # print()
# # print('Количество ненулевых элементов в тестовой выборке', test_vectors.nnz)
# # print('Процент заполненности матрицы признаков {:.2f}%'.format(test_vectors.nnz * 100 / (test_vectors.shape[0] * test_vectors.shape[1])))

# train_dataset = SparseFeaturesDataset(train_vectors, train_source['target'])
# test_dataset = SparseFeaturesDataset(test_vectors, test_source['target'])

# model = nn.Linear(UNIQUE_WORDS_N, UNIQUE_LABELS_N)

# scheduler = lambda optim: \
#     torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=5, factor=0.5, verbose=True)

# experiment_name = 'MAX_DF=0.60_MIN_COUNT=80_min_token_size=2_tflidf_lr=1e-2'
# histories[experiment_name], best_models[experiment_name] = train_eval_loop(
#                                             model=model,
#                                             train_dataset=train_dataset,
#                                             val_dataset=test_dataset,
#                                             criterion=F.cross_entropy,
#                                             lr=1e-2,
#                                             epoch_n=200,
#                                             batch_size=32,
#                                             l2_reg_alpha=0,
#                                             lr_scheduler_ctor=scheduler,
#                                             best_acc_type = 'acc',
#                                             test_dataset=test_dataset,
#                                             experiment_name = experiment_name)

In [ ]:
# show_experiments_stats(histories, show_plots = False)

In [ ]:
#ниже изучение параметров частотности словаря и распеределения по по количеству токенов

In [ ]:
# train_tokenized = tokenize_corpus(train_source['data'], min_token_size=2)
# test_tokenized = tokenize_corpus(test_source['data'], min_token_size=2)

# print(' '.join(train_tokenized[0]))

In [ ]:
# MAX_DF = 1
# MIN_COUNT = 0
# vocabulary, word_doc_freq = build_vocabulary(train_tokenized, max_doc_freq=MAX_DF, min_count=MIN_COUNT)
# UNIQUE_WORDS_N = len(vocabulary)
# print('Количество уникальных токенов', UNIQUE_WORDS_N)
# print(list(vocabulary.items())[:10])

In [ ]:
# print(list(vocabulary.items())[:20])
# word_doc_freq[:20]

In [ ]:
# print(list(vocabulary.items())[-20:])
# word_doc_freq[-20:]

In [ ]:
# plt.hist(word_doc_freq, bins=20)
# plt.title('Распределение относительных частот слов')
# plt.yscale('log');

### Эксперименты с архитектурой нейросети

In [ ]:
train_tokenized = tokenize_corpus(train_source['data'], min_token_size=4)
test_tokenized = tokenize_corpus(test_source['data'], min_token_size=4)

print(' '.join(train_tokenized[0]))

MAX_DF = 0.80
MIN_COUNT = 5

vocabulary, word_doc_freq = build_vocabulary(train_tokenized, max_doc_freq=MAX_DF, min_count=MIN_COUNT)
UNIQUE_WORDS_N = len(vocabulary)
print('Количество уникальных токенов', UNIQUE_WORDS_N)
print(list(vocabulary.items())[:10])

# word_doc_freq[:20]

# word_doc_freq[-20:]

# plt.hist(word_doc_freq, bins=20)
# plt.title('Распределение относительных частот слов')
# plt.yscale('log');

VECTORIZATION_MODE = 'tflidf'
train_vectors = vectorize_texts(train_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)
test_vectors = vectorize_texts(test_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)

print('Размерность матрицы признаков обучающей выборки', train_vectors.shape)
print('Размерность матрицы признаков тестовой выборки', test_vectors.shape)
print()
print('Количество ненулевых элементов в обучающей выборке', train_vectors.nnz)
print('Процент заполненности матрицы признаков {:.2f}%'.format(train_vectors.nnz * 100 / (train_vectors.shape[0] * train_vectors.shape[1])))
print()
print('Количество ненулевых элементов в тестовой выборке', test_vectors.nnz)
print('Процент заполненности матрицы признаков {:.2f}%'.format(test_vectors.nnz * 100 / (test_vectors.shape[0] * test_vectors.shape[1])))

# plt.hist(train_vectors.data, bins=20)
# plt.title('Распределение весов признаков')
# plt.yscale('log');

train_dataset = SparseFeaturesDataset(train_vectors, train_source['target'])
test_dataset = SparseFeaturesDataset(test_vectors, test_source['target'])

In [ ]:
# hidden01_neurons = 240
# dropout_p = 0.6
# model = torch.nn.Sequential(nn.Linear(UNIQUE_WORDS_N, hidden01_neurons),
#                             torch.nn.Dropout(dropout_p),
#                             torch.nn.ReLU(),
#                             nn.Linear(hidden01_neurons, UNIQUE_LABELS_N))

# scheduler = lambda optim: \
#     torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=5, factor=0.5, verbose=True)

# experiment_name = 'NN_Linear(240)_ReLU_Dropout(0.6)__tflidf_lr=1e-2'
# histories[experiment_name], best_models[experiment_name] = train_eval_loop(
#                                             model=model,
#                                             train_dataset=train_dataset,
#                                             val_dataset=test_dataset,
#                                             criterion=F.cross_entropy,
#                                             lr=1e-2,
#                                             epoch_n=200,
#                                             batch_size=32,
#                                             l2_reg_alpha=0,
#                                             lr_scheduler_ctor=scheduler,
#                                             best_acc_type = 'acc',
#                                             test_dataset=test_dataset,
#                                             experiment_name = experiment_name)


In [ ]:
# show_experiments_stats(histories, show_plots = False)

### Эксперимент - изменить способ стандартизации данных

In [ ]:
# train_tokenized = tokenize_corpus(train_source['data'], min_token_size=4)
# test_tokenized = tokenize_corpus(test_source['data'], min_token_size=4)

# print(' '.join(train_tokenized[0]))

# MAX_DF = 0.80
# MIN_COUNT = 5

# vocabulary, word_doc_freq = build_vocabulary(train_tokenized, max_doc_freq=MAX_DF, min_count=MIN_COUNT)
# UNIQUE_WORDS_N = len(vocabulary)
# print('Количество уникальных токенов', UNIQUE_WORDS_N)
# print(list(vocabulary.items())[:10])

# # word_doc_freq[:20]

# # word_doc_freq[-20:]

# # plt.hist(word_doc_freq, bins=20)
# # plt.title('Распределение относительных частот слов')
# # plt.yscale('log');

# VECTORIZATION_MODE = 'tflidf'
# train_vectors = vectorize_texts(train_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)
# test_vectors = vectorize_texts(test_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)

# print('Размерность матрицы признаков обучающей выборки', train_vectors.shape)
# print('Размерность матрицы признаков тестовой выборки', test_vectors.shape)
# print()
# print('Количество ненулевых элементов в обучающей выборке', train_vectors.nnz)
# print('Процент заполненности матрицы признаков {:.2f}%'.format(train_vectors.nnz * 100 / (train_vectors.shape[0] * train_vectors.shape[1])))
# print()
# print('Количество ненулевых элементов в тестовой выборке', test_vectors.nnz)
# print('Процент заполненности матрицы признаков {:.2f}%'.format(test_vectors.nnz * 100 / (test_vectors.shape[0] * test_vectors.shape[1])))

# # plt.hist(train_vectors.data, bins=20)
# # plt.title('Распределение весов признаков')
# # plt.yscale('log');

In [ ]:
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler(with_mean=False)
# train_vectors = scaler.fit_transform(train_vectors)
# test_vectors = scaler.transform(test_vectors)

In [ ]:
# train_dataset = SparseFeaturesDataset(train_vectors, train_source['target'])
# test_dataset = SparseFeaturesDataset(test_vectors, test_source['target'])

In [ ]:
# model = nn.Linear(UNIQUE_WORDS_N, UNIQUE_LABELS_N)

# scheduler = lambda optim: \
#     torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=5, factor=0.5, verbose=True)

# experiment_name = 'StandardScaler__tflidf_lr=1e-2'
# histories[experiment_name], best_models[experiment_name] = train_eval_loop(
#                                             model=model,
#                                             train_dataset=train_dataset,
#                                             val_dataset=test_dataset,
#                                             criterion=F.cross_entropy,
#                                             lr=1e-2,
#                                             epoch_n=200,
#                                             batch_size=32,
#                                             l2_reg_alpha=0,
#                                             lr_scheduler_ctor=scheduler,
#                                             best_acc_type = 'acc',
#                                             test_dataset=test_dataset,
#                                             experiment_name = experiment_name)


In [ ]:
# show_experiments_stats(histories, show_plots = False)

### Эксперимент - извлекать признаки не через токены, а через N-граммы

In [ ]:
# from sklearn.feature_extraction.text import TfidfVectorizer

# MAX_DF = 0.8
# MIN_COUNT = 5
# NGRAM_RANGE=(2,2)

# vectorizer = TfidfVectorizer(analyzer='word', ngram_range=NGRAM_RANGE, max_df=MAX_DF, min_df=MIN_COUNT)
# train_vectors = vectorizer.fit_transform(train_source['data'])
# test_vectors = vectorizer.transform(test_source['data'])

# print('Размерность матрицы признаков обучающей выборки', train_vectors.shape)
# print('Размерность матрицы признаков тестовой выборки', test_vectors.shape)
# print()
# print('Количество ненулевых элементов в обучающей выборке', train_vectors.nnz)
# print('Процент заполненности матрицы признаков {:.2f}%'.format(train_vectors.nnz * 100 / (train_vectors.shape[0] * train_vectors.shape[1])))
# print()
# print('Количество ненулевых элементов в тестовой выборке', test_vectors.nnz)
# print('Процент заполненности матрицы признаков {:.2f}%'.format(test_vectors.nnz * 100 / (test_vectors.shape[0] * test_vectors.shape[1])))

# train_dataset = SparseFeaturesDataset(train_vectors, train_source['target'])
# test_dataset = SparseFeaturesDataset(test_vectors, test_source['target'])

# UNIQUE_WORDS_N = train_vectors.shape[-1]

# model = nn.Linear(UNIQUE_WORDS_N, UNIQUE_LABELS_N)

# scheduler = lambda optim: \
#     torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=5, factor=0.5, verbose=True)

# experiment_name = 'NGRAM_RANGE=(2,2)_MIN_COUNT=5__sklearn.tfidfVectorizer_lr=1e-2'
# histories[experiment_name], best_models[experiment_name] = train_eval_loop(
#                                             model=model,
#                                             train_dataset=train_dataset,
#                                             val_dataset=test_dataset,
#                                             criterion=F.cross_entropy,
#                                             lr=1e-2,
#                                             epoch_n=200,
#                                             batch_size=32,
#                                             l2_reg_alpha=0,
#                                             lr_scheduler_ctor=scheduler,
#                                             best_acc_type = 'acc',
#                                             test_dataset=test_dataset,
#                                             experiment_name = experiment_name)


In [ ]:

# MAX_DF = 0.8
# MIN_COUNT = 5
# NGRAM_RANGE=(1,2)

# vectorizer = TfidfVectorizer(analyzer='word', ngram_range=NGRAM_RANGE, max_df=MAX_DF, min_df=MIN_COUNT)
# train_vectors = vectorizer.fit_transform(train_source['data'])
# test_vectors = vectorizer.transform(test_source['data'])

# print('Размерность матрицы признаков обучающей выборки', train_vectors.shape)
# print('Размерность матрицы признаков тестовой выборки', test_vectors.shape)
# print()
# print('Количество ненулевых элементов в обучающей выборке', train_vectors.nnz)
# print('Процент заполненности матрицы признаков {:.2f}%'.format(train_vectors.nnz * 100 / (train_vectors.shape[0] * train_vectors.shape[1])))
# print()
# print('Количество ненулевых элементов в тестовой выборке', test_vectors.nnz)
# print('Процент заполненности матрицы признаков {:.2f}%'.format(test_vectors.nnz * 100 / (test_vectors.shape[0] * test_vectors.shape[1])))

# train_dataset = SparseFeaturesDataset(train_vectors, train_source['target'])
# test_dataset = SparseFeaturesDataset(test_vectors, test_source['target'])

# UNIQUE_WORDS_N = train_vectors.shape[-1]

# model = nn.Linear(UNIQUE_WORDS_N, UNIQUE_LABELS_N)

# scheduler = lambda optim: \
#     torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=5, factor=0.5, verbose=True)

# experiment_name = 'NGRAM_RANGE=(1,2)_MIN_COUNT=5__sklearn.tfidfVectorizer_lr=1e-2'
# histories[experiment_name], best_models[experiment_name] = train_eval_loop(
#                                             model=model,
#                                             train_dataset=train_dataset,
#                                             val_dataset=test_dataset,
#                                             criterion=F.cross_entropy,
#                                             lr=1e-2,
#                                             epoch_n=200,
#                                             batch_size=32,
#                                             l2_reg_alpha=0,
#                                             lr_scheduler_ctor=scheduler,
#                                             best_acc_type = 'acc',
#                                             test_dataset=test_dataset,
#                                             experiment_name = experiment_name)


In [ ]:

# MAX_DF = 0.8
# MIN_COUNT = 2
# NGRAM_RANGE=(1,2)

# vectorizer = TfidfVectorizer(analyzer='word', ngram_range=NGRAM_RANGE, max_df=MAX_DF, min_df=MIN_COUNT)
# train_vectors = vectorizer.fit_transform(train_source['data'])
# test_vectors = vectorizer.transform(test_source['data'])

# print('Размерность матрицы признаков обучающей выборки', train_vectors.shape)
# print('Размерность матрицы признаков тестовой выборки', test_vectors.shape)
# print()
# print('Количество ненулевых элементов в обучающей выборке', train_vectors.nnz)
# print('Процент заполненности матрицы признаков {:.2f}%'.format(train_vectors.nnz * 100 / (train_vectors.shape[0] * train_vectors.shape[1])))
# print()
# print('Количество ненулевых элементов в тестовой выборке', test_vectors.nnz)
# print('Процент заполненности матрицы признаков {:.2f}%'.format(test_vectors.nnz * 100 / (test_vectors.shape[0] * test_vectors.shape[1])))

# train_dataset = SparseFeaturesDataset(train_vectors, train_source['target'])
# test_dataset = SparseFeaturesDataset(test_vectors, test_source['target'])

# UNIQUE_WORDS_N = train_vectors.shape[-1]

# model = nn.Linear(UNIQUE_WORDS_N, UNIQUE_LABELS_N)

# scheduler = lambda optim: \
#     torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=5, factor=0.5, verbose=True)

# experiment_name = 'NGRAM_RANGE=(1,2)_MIN_COUNT=2__sklearn.tfidfVectorizer_lr=1e-2'
# histories[experiment_name], best_models[experiment_name] = train_eval_loop(
#                                             model=model,
#                                             train_dataset=train_dataset,
#                                             val_dataset=test_dataset,
#                                             criterion=F.cross_entropy,
#                                             lr=1e-2,
#                                             epoch_n=200,
#                                             batch_size=32,
#                                             l2_reg_alpha=0,
#                                             lr_scheduler_ctor=scheduler,
#                                             best_acc_type = 'acc',
#                                             test_dataset=test_dataset,
#                                             experiment_name = experiment_name)


In [ ]:
# MAX_DF = 0.8
# MIN_COUNT = 2
# NGRAM_RANGE=(1,3)

# vectorizer = TfidfVectorizer(analyzer='word', ngram_range=NGRAM_RANGE, max_df=MAX_DF, min_df=MIN_COUNT)
# train_vectors = vectorizer.fit_transform(train_source['data'])
# test_vectors = vectorizer.transform(test_source['data'])

# print('Размерность матрицы признаков обучающей выборки', train_vectors.shape)
# print('Размерность матрицы признаков тестовой выборки', test_vectors.shape)
# print()
# print('Количество ненулевых элементов в обучающей выборке', train_vectors.nnz)
# print('Процент заполненности матрицы признаков {:.2f}%'.format(train_vectors.nnz * 100 / (train_vectors.shape[0] * train_vectors.shape[1])))
# print()
# print('Количество ненулевых элементов в тестовой выборке', test_vectors.nnz)
# print('Процент заполненности матрицы признаков {:.2f}%'.format(test_vectors.nnz * 100 / (test_vectors.shape[0] * test_vectors.shape[1])))

# train_dataset = SparseFeaturesDataset(train_vectors, train_source['target'])
# test_dataset = SparseFeaturesDataset(test_vectors, test_source['target'])

# UNIQUE_WORDS_N = train_vectors.shape[-1]

# model = nn.Linear(UNIQUE_WORDS_N, UNIQUE_LABELS_N)

# scheduler = lambda optim: \
#     torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=5, factor=0.5, verbose=True)

# experiment_name = 'NGRAM_RANGE=(1,3)_MIN_COUNT=2__sklearn.tfidfVectorizer_lr=1e-2'
# histories[experiment_name], best_models[experiment_name] = train_eval_loop(
#                                             model=model,
#                                             train_dataset=train_dataset,
#                                             val_dataset=test_dataset,
#                                             criterion=F.cross_entropy,
#                                             lr=1e-2,
#                                             epoch_n=200,
#                                             batch_size=32,
#                                             l2_reg_alpha=0,
#                                             lr_scheduler_ctor=scheduler,
#                                             best_acc_type = 'acc',
#                                             test_dataset=test_dataset,
#                                             experiment_name = experiment_name)


In [ ]:
# show_experiments_stats(histories, show_plots = False)

### Эксперимент - взвешивание признаков с помощью точечной взаимной информации (PMI)

In [ ]:
# train_tokenized = tokenize_corpus(train_source['data'], min_token_size=2)
# test_tokenized = tokenize_corpus(test_source['data'], min_token_size=2)

# print(' '.join(train_tokenized[0]))

In [ ]:
# PMI_TYPE = 'max'
# PMI_Q_THRESHOLD = 1.0
# MIN_COUNT = 5
# min_token_size=2
# NO_NEG = False
# PWL_DENOM = 'size(c)'

# vocabulary, word_doc_pmi = build_vocabulary_pmi(train_tokenized, train_source['target'], 
#                                                 min_count = MIN_COUNT, pmi_type = PMI_TYPE, 
#                                                 invers_quantile_threshold = PMI_Q_THRESHOLD, 
#                                                 no_negative = NO_NEG, p_w_l_denominator = PWL_DENOM)
# UNIQUE_WORDS_N = len(vocabulary)
# print('Количество уникальных токенов', UNIQUE_WORDS_N)
# print(list(vocabulary.items())[:10])

# # word_doc_pmi[:20]

# # word_doc_pmi[-20:]

# # plt.hist(word_doc_pmi, bins=20)
# # plt.title('Распределение относительных частот слов')
# # plt.yscale('log');

# VECTORIZATION_MODE = 'tfpmi'
# train_vectors = vectorize_texts(train_tokenized, vocabulary, word_doc_pmi, mode=VECTORIZATION_MODE)
# test_vectors = vectorize_texts(test_tokenized, vocabulary, word_doc_pmi, mode=VECTORIZATION_MODE)

# print('Размерность матрицы признаков обучающей выборки', train_vectors.shape)
# print('Размерность матрицы признаков тестовой выборки', test_vectors.shape)
# print()
# print('Количество ненулевых элементов в обучающей выборке', train_vectors.nnz)
# print('Процент заполненности матрицы признаков {:.2f}%'.format(train_vectors.nnz * 100 / (train_vectors.shape[0] * train_vectors.shape[1])))
# print()
# print('Количество ненулевых элементов в тестовой выборке', test_vectors.nnz)
# print('Процент заполненности матрицы признаков {:.2f}%'.format(test_vectors.nnz * 100 / (test_vectors.shape[0] * test_vectors.shape[1])))

# # plt.hist(train_vectors.data, bins=20)
# # plt.title('Распределение весов признаков')
# # plt.yscale('log');

# train_dataset = SparseFeaturesDataset(train_vectors, train_source['target'])
# test_dataset = SparseFeaturesDataset(test_vectors, test_source['target'])

# model = nn.Linear(UNIQUE_WORDS_N, UNIQUE_LABELS_N)

# scheduler = lambda optim: \
#     torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=5, factor=0.5, verbose=True)

# experiment_name = 'PMI_TYPE=max_PMI_Q_THRESHOLD=1.0_MIN_COUNT=_5_PWL_DENOM=size(c)_min_token_size=2_tf_lr=1e-2'
# histories[experiment_name], best_models[experiment_name] = train_eval_loop(
#                                             model=model,
#                                             train_dataset=train_dataset,
#                                             val_dataset=test_dataset,
#                                             criterion=F.cross_entropy,
#                                             lr=1e-2,
#                                             epoch_n=200,
#                                             batch_size=32,
#                                             l2_reg_alpha=0,
#                                             lr_scheduler_ctor=scheduler,
#                                             best_acc_type = 'acc',
#                                             test_dataset=test_dataset,
#                                             experiment_name = experiment_name)


In [ ]:
# show_experiments_stats(histories, show_plots = False)

### Финальные эксперименты (микс из лучшего)

In [ ]:
min_token_size=2
ADD_STEMMS = True
MIN_COUNT = 2
VECTORIZATION_MODE = 'tflidf'

if 'pmi' in VECTORIZATION_MODE:
    MAX_QUAN = 0.8
    MAX_TYPE = 'MAX_QUAN={}'.format(MAX_QUAN) #only for generate experiment_name
else:
    MAX_DF = 0.8
    MAX_TYPE = 'MAX_DF={}'.format(MAX_DF) #only for generate experiment_name

HIDDEN_NEURONS = 240
DROPOUT_p = 0.4
BATCH_SIZE = 64
LR = 1e-2
L2REG = 0
SCHED_PAT = 4

experiment_name = 'mtoken_size={}_MIN_COUNT={}_ADD_STEMMS_{}_{}_NN_Linear={}_Dropout={}_lr={}_BATCH={}_L2REG={}_SCHED_PAT={}'.format(
                    min_token_size, MIN_COUNT, 
                    MAX_TYPE,
                    VECTORIZATION_MODE, HIDDEN_NEURONS, DROPOUT_p, LR, BATCH_SIZE, L2REG, SCHED_PAT)


train_tokenized = tokenize_corpus(train_source['data'], min_token_size=2)
test_tokenized = tokenize_corpus(test_source['data'], min_token_size=2)

print(' '.join(train_tokenized[0]))

if ADD_STEMMS:
    #PorterStemmer:
    from nltk.stem.porter import PorterStemmer
    ps = PorterStemmer()
    converter = ps.stem

    train_tokenized = tokenize_corpus_convert(train_tokenized, converter=converter, addition = True)
    test_tokenized = tokenize_corpus_convert(test_tokenized, converter=converter, addition = True)

    print(' '.join(train_tokenized[0]))

    
if 'pmi' in VECTORIZATION_MODE:
    vocabulary, word_doc_pmi = build_vocabulary_pmi(train_tokenized, train_source['target'], 
                                                invers_quantile_threshold = MAX_QUAN, min_count = MIN_COUNT)
    
    train_vectors = vectorize_texts(train_tokenized, vocabulary, word_doc_pmi, mode=VECTORIZATION_MODE)
    test_vectors = vectorize_texts(test_tokenized, vocabulary, word_doc_pmi, mode=VECTORIZATION_MODE)
else:
    vocabulary, word_doc_freq = build_vocabulary(train_tokenized, max_doc_freq=MAX_DF, min_count=MIN_COUNT)
    
    train_vectors = vectorize_texts(train_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)
    test_vectors = vectorize_texts(test_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)



UNIQUE_WORDS_N = len(vocabulary)
print('Количество уникальных токенов', UNIQUE_WORDS_N)
print(list(vocabulary.items())[:10])

print('Размерность матрицы признаков обучающей выборки', train_vectors.shape)
print('Размерность матрицы признаков тестовой выборки', test_vectors.shape)
print()
print('Количество ненулевых элементов в обучающей выборке', train_vectors.nnz)
print('Процент заполненности матрицы признаков {:.2f}%'.format(train_vectors.nnz * 100 / (train_vectors.shape[0] * train_vectors.shape[1])))
print()
print('Количество ненулевых элементов в тестовой выборке', test_vectors.nnz)
print('Процент заполненности матрицы признаков {:.2f}%'.format(test_vectors.nnz * 100 / (test_vectors.shape[0] * test_vectors.shape[1])))

UNIQUE_LABELS_N = len(set(train_source['target']))
print('Количество уникальных меток', UNIQUE_LABELS_N)
set(train_source['target'])

train_dataset = SparseFeaturesDataset(train_vectors, train_source['target'])
test_dataset = SparseFeaturesDataset(test_vectors, test_source['target'])

model = torch.nn.Sequential(nn.Linear(UNIQUE_WORDS_N, HIDDEN_NEURONS),
                            torch.nn.Dropout(DROPOUT_p),
                            torch.nn.ReLU(),
                            nn.Linear(HIDDEN_NEURONS, UNIQUE_LABELS_N))

scheduler = lambda optim: \
    torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=SCHED_PAT, factor=0.5, verbose=True)

histories[experiment_name], best_models[experiment_name] = train_eval_loop(
                                            model=model,
                                            train_dataset=train_dataset,
                                            val_dataset=test_dataset,
                                            criterion=F.cross_entropy,
                                            lr=LR,
                                            epoch_n=200,
                                            batch_size=BATCH_SIZE,
                                            l2_reg_alpha=L2REG,
                                            lr_scheduler_ctor=scheduler,
                                            best_acc_type = 'acc',
                                            test_dataset=test_dataset,
                                            early_stopping_patience=6,
                                            experiment_name = experiment_name)

In [ ]:
min_token_size=2
ADD_STEMMS = True
MIN_COUNT = 2
VECTORIZATION_MODE = 'tflidf'

if 'pmi' in VECTORIZATION_MODE:
    MAX_QUAN = 0.8
    MAX_TYPE = 'MAX_QUAN={}'.format(MAX_QUAN) #only for generate experiment_name
else:
    MAX_DF = 0.8
    MAX_TYPE = 'MAX_DF={}'.format(MAX_DF) #only for generate experiment_name

HIDDEN_NEURONS = 340
DROPOUT_p = 0.4
BATCH_SIZE = 64
LR = 1e-2
L2REG = 0
SCHED_PAT = 4

experiment_name = 'mtoken_size={}_MIN_COUNT={}_ADD_STEMMS_{}_{}_NN_Linear={}_Dropout={}_lr={}_BATCH={}_L2REG={}_SCHED_PAT={}'.format(
                    min_token_size, MIN_COUNT, 
                    MAX_TYPE,
                    VECTORIZATION_MODE, HIDDEN_NEURONS, DROPOUT_p, LR, BATCH_SIZE, L2REG, SCHED_PAT)


train_tokenized = tokenize_corpus(train_source['data'], min_token_size=2)
test_tokenized = tokenize_corpus(test_source['data'], min_token_size=2)

print(' '.join(train_tokenized[0]))

if ADD_STEMMS:
    #PorterStemmer:
    from nltk.stem.porter import PorterStemmer
    ps = PorterStemmer()
    converter = ps.stem

    train_tokenized = tokenize_corpus_convert(train_tokenized, converter=converter, addition = True)
    test_tokenized = tokenize_corpus_convert(test_tokenized, converter=converter, addition = True)

    print(' '.join(train_tokenized[0]))

    
if 'pmi' in VECTORIZATION_MODE:
    vocabulary, word_doc_pmi = build_vocabulary_pmi(train_tokenized, train_source['target'], 
                                                invers_quantile_threshold = MAX_QUAN, min_count = MIN_COUNT)
    
    train_vectors = vectorize_texts(train_tokenized, vocabulary, word_doc_pmi, mode=VECTORIZATION_MODE)
    test_vectors = vectorize_texts(test_tokenized, vocabulary, word_doc_pmi, mode=VECTORIZATION_MODE)
else:
    vocabulary, word_doc_freq = build_vocabulary(train_tokenized, max_doc_freq=MAX_DF, min_count=MIN_COUNT)
    
    train_vectors = vectorize_texts(train_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)
    test_vectors = vectorize_texts(test_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)



UNIQUE_WORDS_N = len(vocabulary)
print('Количество уникальных токенов', UNIQUE_WORDS_N)
print(list(vocabulary.items())[:10])

print('Размерность матрицы признаков обучающей выборки', train_vectors.shape)
print('Размерность матрицы признаков тестовой выборки', test_vectors.shape)
print()
print('Количество ненулевых элементов в обучающей выборке', train_vectors.nnz)
print('Процент заполненности матрицы признаков {:.2f}%'.format(train_vectors.nnz * 100 / (train_vectors.shape[0] * train_vectors.shape[1])))
print()
print('Количество ненулевых элементов в тестовой выборке', test_vectors.nnz)
print('Процент заполненности матрицы признаков {:.2f}%'.format(test_vectors.nnz * 100 / (test_vectors.shape[0] * test_vectors.shape[1])))

UNIQUE_LABELS_N = len(set(train_source['target']))
print('Количество уникальных меток', UNIQUE_LABELS_N)
set(train_source['target'])

train_dataset = SparseFeaturesDataset(train_vectors, train_source['target'])
test_dataset = SparseFeaturesDataset(test_vectors, test_source['target'])

model = torch.nn.Sequential(nn.Linear(UNIQUE_WORDS_N, HIDDEN_NEURONS),
                            torch.nn.Dropout(DROPOUT_p),
                            torch.nn.ReLU(),
                            nn.Linear(HIDDEN_NEURONS, UNIQUE_LABELS_N))

scheduler = lambda optim: \
    torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=SCHED_PAT, factor=0.5, verbose=True)

histories[experiment_name], best_models[experiment_name] = train_eval_loop(
                                            model=model,
                                            train_dataset=train_dataset,
                                            val_dataset=test_dataset,
                                            criterion=F.cross_entropy,
                                            lr=LR,
                                            epoch_n=200,
                                            batch_size=BATCH_SIZE,
                                            l2_reg_alpha=L2REG,
                                            lr_scheduler_ctor=scheduler,
                                            best_acc_type = 'acc',
                                            test_dataset=test_dataset,
                                            early_stopping_patience=6,
                                            experiment_name = experiment_name)

In [ ]:
min_token_size=2
ADD_STEMMS = True
MIN_COUNT = 2
VECTORIZATION_MODE = 'tflidf'

if 'pmi' in VECTORIZATION_MODE:
    MAX_QUAN = 0.8
    MAX_TYPE = 'MAX_QUAN={}'.format(MAX_QUAN) #only for generate experiment_name
else:
    MAX_DF = 0.8
    MAX_TYPE = 'MAX_DF={}'.format(MAX_DF) #only for generate experiment_name

HIDDEN_NEURONS = 400
DROPOUT_p = 0.4
BATCH_SIZE = 64
LR = 1e-2
L2REG = 0
SCHED_PAT = 4

experiment_name = 'mtoken_size={}_MIN_COUNT={}_ADD_STEMMS_{}_{}_NN_Linear={}_Dropout={}_lr={}_BATCH={}_L2REG={}_SCHED_PAT={}'.format(
                    min_token_size, MIN_COUNT, 
                    MAX_TYPE,
                    VECTORIZATION_MODE, HIDDEN_NEURONS, DROPOUT_p, LR, BATCH_SIZE, L2REG, SCHED_PAT)


train_tokenized = tokenize_corpus(train_source['data'], min_token_size=2)
test_tokenized = tokenize_corpus(test_source['data'], min_token_size=2)

print(' '.join(train_tokenized[0]))

if ADD_STEMMS:
    #PorterStemmer:
    from nltk.stem.porter import PorterStemmer
    ps = PorterStemmer()
    converter = ps.stem

    train_tokenized = tokenize_corpus_convert(train_tokenized, converter=converter, addition = True)
    test_tokenized = tokenize_corpus_convert(test_tokenized, converter=converter, addition = True)

    print(' '.join(train_tokenized[0]))

    
if 'pmi' in VECTORIZATION_MODE:
    vocabulary, word_doc_pmi = build_vocabulary_pmi(train_tokenized, train_source['target'], 
                                                invers_quantile_threshold = MAX_QUAN, min_count = MIN_COUNT)
    
    train_vectors = vectorize_texts(train_tokenized, vocabulary, word_doc_pmi, mode=VECTORIZATION_MODE)
    test_vectors = vectorize_texts(test_tokenized, vocabulary, word_doc_pmi, mode=VECTORIZATION_MODE)
else:
    vocabulary, word_doc_freq = build_vocabulary(train_tokenized, max_doc_freq=MAX_DF, min_count=MIN_COUNT)
    
    train_vectors = vectorize_texts(train_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)
    test_vectors = vectorize_texts(test_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)



UNIQUE_WORDS_N = len(vocabulary)
print('Количество уникальных токенов', UNIQUE_WORDS_N)
print(list(vocabulary.items())[:10])

print('Размерность матрицы признаков обучающей выборки', train_vectors.shape)
print('Размерность матрицы признаков тестовой выборки', test_vectors.shape)
print()
print('Количество ненулевых элементов в обучающей выборке', train_vectors.nnz)
print('Процент заполненности матрицы признаков {:.2f}%'.format(train_vectors.nnz * 100 / (train_vectors.shape[0] * train_vectors.shape[1])))
print()
print('Количество ненулевых элементов в тестовой выборке', test_vectors.nnz)
print('Процент заполненности матрицы признаков {:.2f}%'.format(test_vectors.nnz * 100 / (test_vectors.shape[0] * test_vectors.shape[1])))

UNIQUE_LABELS_N = len(set(train_source['target']))
print('Количество уникальных меток', UNIQUE_LABELS_N)
set(train_source['target'])

train_dataset = SparseFeaturesDataset(train_vectors, train_source['target'])
test_dataset = SparseFeaturesDataset(test_vectors, test_source['target'])

model = torch.nn.Sequential(nn.Linear(UNIQUE_WORDS_N, HIDDEN_NEURONS),
                            torch.nn.Dropout(DROPOUT_p),
                            torch.nn.ReLU(),
                            nn.Linear(HIDDEN_NEURONS, UNIQUE_LABELS_N))

scheduler = lambda optim: \
    torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=SCHED_PAT, factor=0.5, verbose=True)

histories[experiment_name], best_models[experiment_name] = train_eval_loop(
                                            model=model,
                                            train_dataset=train_dataset,
                                            val_dataset=test_dataset,
                                            criterion=F.cross_entropy,
                                            lr=LR,
                                            epoch_n=200,
                                            batch_size=BATCH_SIZE,
                                            l2_reg_alpha=L2REG,
                                            lr_scheduler_ctor=scheduler,
                                            best_acc_type = 'acc',
                                            test_dataset=test_dataset,
                                            early_stopping_patience=6,
                                            experiment_name = experiment_name)

In [ ]:
min_token_size=2
ADD_STEMMS = True
MIN_COUNT = 2
VECTORIZATION_MODE = 'tflidf'

if 'pmi' in VECTORIZATION_MODE:
    MAX_QUAN = 0.8
    MAX_TYPE = 'MAX_QUAN={}'.format(MAX_QUAN) #only for generate experiment_name
else:
    MAX_DF = 0.98
    MAX_TYPE = 'MAX_DF={}'.format(MAX_DF) #only for generate experiment_name

HIDDEN_NEURONS = 240
DROPOUT_p = 0.4
BATCH_SIZE = 64
LR = 1e-2
L2REG = 0
SCHED_PAT = 4

experiment_name = 'mtoken_size={}_MIN_COUNT={}_ADD_STEMMS_{}_{}_NN_Linear={}_Dropout={}_lr={}_BATCH={}_L2REG={}_SCHED_PAT={}'.format(
                    min_token_size, MIN_COUNT, 
                    MAX_TYPE,
                    VECTORIZATION_MODE, HIDDEN_NEURONS, DROPOUT_p, LR, BATCH_SIZE, L2REG, SCHED_PAT)


train_tokenized = tokenize_corpus(train_source['data'], min_token_size=2)
test_tokenized = tokenize_corpus(test_source['data'], min_token_size=2)

print(' '.join(train_tokenized[0]))

if ADD_STEMMS:
    #PorterStemmer:
    from nltk.stem.porter import PorterStemmer
    ps = PorterStemmer()
    converter = ps.stem

    train_tokenized = tokenize_corpus_convert(train_tokenized, converter=converter, addition = True)
    test_tokenized = tokenize_corpus_convert(test_tokenized, converter=converter, addition = True)

    print(' '.join(train_tokenized[0]))

    
if 'pmi' in VECTORIZATION_MODE:
    vocabulary, word_doc_pmi = build_vocabulary_pmi(train_tokenized, train_source['target'], 
                                                invers_quantile_threshold = MAX_QUAN, min_count = MIN_COUNT)
    
    train_vectors = vectorize_texts(train_tokenized, vocabulary, word_doc_pmi, mode=VECTORIZATION_MODE)
    test_vectors = vectorize_texts(test_tokenized, vocabulary, word_doc_pmi, mode=VECTORIZATION_MODE)
else:
    vocabulary, word_doc_freq = build_vocabulary(train_tokenized, max_doc_freq=MAX_DF, min_count=MIN_COUNT)
    
    train_vectors = vectorize_texts(train_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)
    test_vectors = vectorize_texts(test_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)



UNIQUE_WORDS_N = len(vocabulary)
print('Количество уникальных токенов', UNIQUE_WORDS_N)
print(list(vocabulary.items())[:10])

print('Размерность матрицы признаков обучающей выборки', train_vectors.shape)
print('Размерность матрицы признаков тестовой выборки', test_vectors.shape)
print()
print('Количество ненулевых элементов в обучающей выборке', train_vectors.nnz)
print('Процент заполненности матрицы признаков {:.2f}%'.format(train_vectors.nnz * 100 / (train_vectors.shape[0] * train_vectors.shape[1])))
print()
print('Количество ненулевых элементов в тестовой выборке', test_vectors.nnz)
print('Процент заполненности матрицы признаков {:.2f}%'.format(test_vectors.nnz * 100 / (test_vectors.shape[0] * test_vectors.shape[1])))

UNIQUE_LABELS_N = len(set(train_source['target']))
print('Количество уникальных меток', UNIQUE_LABELS_N)
set(train_source['target'])

train_dataset = SparseFeaturesDataset(train_vectors, train_source['target'])
test_dataset = SparseFeaturesDataset(test_vectors, test_source['target'])

model = torch.nn.Sequential(nn.Linear(UNIQUE_WORDS_N, HIDDEN_NEURONS),
                            torch.nn.Dropout(DROPOUT_p),
                            torch.nn.ReLU(),
                            nn.Linear(HIDDEN_NEURONS, UNIQUE_LABELS_N))

scheduler = lambda optim: \
    torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=SCHED_PAT, factor=0.5, verbose=True)

histories[experiment_name], best_models[experiment_name] = train_eval_loop(
                                            model=model,
                                            train_dataset=train_dataset,
                                            val_dataset=test_dataset,
                                            criterion=F.cross_entropy,
                                            lr=LR,
                                            epoch_n=200,
                                            batch_size=BATCH_SIZE,
                                            l2_reg_alpha=L2REG,
                                            lr_scheduler_ctor=scheduler,
                                            best_acc_type = 'acc',
                                            test_dataset=test_dataset,
                                            early_stopping_patience=6,
                                            experiment_name = experiment_name)

In [ ]:
min_token_size=2
ADD_STEMMS = True
MIN_COUNT = 2
VECTORIZATION_MODE = 'tflidf'

if 'pmi' in VECTORIZATION_MODE:
    MAX_QUAN = 0.8
    MAX_TYPE = 'MAX_QUAN={}'.format(MAX_QUAN) #only for generate experiment_name
else:
    MAX_DF = 0.8
    MAX_TYPE = 'MAX_DF={}'.format(MAX_DF) #only for generate experiment_name

HIDDEN_NEURONS = 240
DROPOUT_p = 0.3
BATCH_SIZE = 64
LR = 1e-2
L2REG = 0
SCHED_PAT = 4

experiment_name = 'mtoken_size={}_MIN_COUNT={}_ADD_STEMMS_{}_{}_NN_Linear={}_Dropout={}_lr={}_BATCH={}_L2REG={}_SCHED_PAT={}'.format(
                    min_token_size, MIN_COUNT, 
                    MAX_TYPE,
                    VECTORIZATION_MODE, HIDDEN_NEURONS, DROPOUT_p, LR, BATCH_SIZE, L2REG, SCHED_PAT)


train_tokenized = tokenize_corpus(train_source['data'], min_token_size=2)
test_tokenized = tokenize_corpus(test_source['data'], min_token_size=2)

print(' '.join(train_tokenized[0]))

if ADD_STEMMS:
    #PorterStemmer:
    from nltk.stem.porter import PorterStemmer
    ps = PorterStemmer()
    converter = ps.stem

    train_tokenized = tokenize_corpus_convert(train_tokenized, converter=converter, addition = True)
    test_tokenized = tokenize_corpus_convert(test_tokenized, converter=converter, addition = True)

    print(' '.join(train_tokenized[0]))

    
if 'pmi' in VECTORIZATION_MODE:
    vocabulary, word_doc_pmi = build_vocabulary_pmi(train_tokenized, train_source['target'], 
                                                invers_quantile_threshold = MAX_QUAN, min_count = MIN_COUNT)
    
    train_vectors = vectorize_texts(train_tokenized, vocabulary, word_doc_pmi, mode=VECTORIZATION_MODE)
    test_vectors = vectorize_texts(test_tokenized, vocabulary, word_doc_pmi, mode=VECTORIZATION_MODE)
else:
    vocabulary, word_doc_freq = build_vocabulary(train_tokenized, max_doc_freq=MAX_DF, min_count=MIN_COUNT)
    
    train_vectors = vectorize_texts(train_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)
    test_vectors = vectorize_texts(test_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)



UNIQUE_WORDS_N = len(vocabulary)
print('Количество уникальных токенов', UNIQUE_WORDS_N)
print(list(vocabulary.items())[:10])

print('Размерность матрицы признаков обучающей выборки', train_vectors.shape)
print('Размерность матрицы признаков тестовой выборки', test_vectors.shape)
print()
print('Количество ненулевых элементов в обучающей выборке', train_vectors.nnz)
print('Процент заполненности матрицы признаков {:.2f}%'.format(train_vectors.nnz * 100 / (train_vectors.shape[0] * train_vectors.shape[1])))
print()
print('Количество ненулевых элементов в тестовой выборке', test_vectors.nnz)
print('Процент заполненности матрицы признаков {:.2f}%'.format(test_vectors.nnz * 100 / (test_vectors.shape[0] * test_vectors.shape[1])))

UNIQUE_LABELS_N = len(set(train_source['target']))
print('Количество уникальных меток', UNIQUE_LABELS_N)
set(train_source['target'])

train_dataset = SparseFeaturesDataset(train_vectors, train_source['target'])
test_dataset = SparseFeaturesDataset(test_vectors, test_source['target'])

model = torch.nn.Sequential(nn.Linear(UNIQUE_WORDS_N, HIDDEN_NEURONS),
                            torch.nn.Dropout(DROPOUT_p),
                            torch.nn.ReLU(),
                            nn.Linear(HIDDEN_NEURONS, UNIQUE_LABELS_N))

scheduler = lambda optim: \
    torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=SCHED_PAT, factor=0.5, verbose=True)

histories[experiment_name], best_models[experiment_name] = train_eval_loop(
                                            model=model,
                                            train_dataset=train_dataset,
                                            val_dataset=test_dataset,
                                            criterion=F.cross_entropy,
                                            lr=LR,
                                            epoch_n=200,
                                            batch_size=BATCH_SIZE,
                                            l2_reg_alpha=L2REG,
                                            lr_scheduler_ctor=scheduler,
                                            best_acc_type = 'acc',
                                            test_dataset=test_dataset,
                                            early_stopping_patience=6,
                                            experiment_name = experiment_name)

In [ ]:
min_token_size=2
ADD_STEMMS = True
MIN_COUNT = 2
VECTORIZATION_MODE = 'tflidf'

if 'pmi' in VECTORIZATION_MODE:
    MAX_QUAN = 0.8
    MAX_TYPE = 'MAX_QUAN={}'.format(MAX_QUAN) #only for generate experiment_name
else:
    MAX_DF = 0.8
    MAX_TYPE = 'MAX_DF={}'.format(MAX_DF) #only for generate experiment_name

HIDDEN_NEURONS = 240
DROPOUT_p = 0.2
BATCH_SIZE = 64
LR = 1e-2
L2REG = 0
SCHED_PAT = 4

experiment_name = 'mtoken_size={}_MIN_COUNT={}_ADD_STEMMS_{}_{}_NN_Linear={}_Dropout={}_lr={}_BATCH={}_L2REG={}_SCHED_PAT={}'.format(
                    min_token_size, MIN_COUNT, 
                    MAX_TYPE,
                    VECTORIZATION_MODE, HIDDEN_NEURONS, DROPOUT_p, LR, BATCH_SIZE, L2REG, SCHED_PAT)


train_tokenized = tokenize_corpus(train_source['data'], min_token_size=2)
test_tokenized = tokenize_corpus(test_source['data'], min_token_size=2)

print(' '.join(train_tokenized[0]))

if ADD_STEMMS:
    #PorterStemmer:
    from nltk.stem.porter import PorterStemmer
    ps = PorterStemmer()
    converter = ps.stem

    train_tokenized = tokenize_corpus_convert(train_tokenized, converter=converter, addition = True)
    test_tokenized = tokenize_corpus_convert(test_tokenized, converter=converter, addition = True)

    print(' '.join(train_tokenized[0]))

    
if 'pmi' in VECTORIZATION_MODE:
    vocabulary, word_doc_pmi = build_vocabulary_pmi(train_tokenized, train_source['target'], 
                                                invers_quantile_threshold = MAX_QUAN, min_count = MIN_COUNT)
    
    train_vectors = vectorize_texts(train_tokenized, vocabulary, word_doc_pmi, mode=VECTORIZATION_MODE)
    test_vectors = vectorize_texts(test_tokenized, vocabulary, word_doc_pmi, mode=VECTORIZATION_MODE)
else:
    vocabulary, word_doc_freq = build_vocabulary(train_tokenized, max_doc_freq=MAX_DF, min_count=MIN_COUNT)
    
    train_vectors = vectorize_texts(train_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)
    test_vectors = vectorize_texts(test_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)



UNIQUE_WORDS_N = len(vocabulary)
print('Количество уникальных токенов', UNIQUE_WORDS_N)
print(list(vocabulary.items())[:10])

print('Размерность матрицы признаков обучающей выборки', train_vectors.shape)
print('Размерность матрицы признаков тестовой выборки', test_vectors.shape)
print()
print('Количество ненулевых элементов в обучающей выборке', train_vectors.nnz)
print('Процент заполненности матрицы признаков {:.2f}%'.format(train_vectors.nnz * 100 / (train_vectors.shape[0] * train_vectors.shape[1])))
print()
print('Количество ненулевых элементов в тестовой выборке', test_vectors.nnz)
print('Процент заполненности матрицы признаков {:.2f}%'.format(test_vectors.nnz * 100 / (test_vectors.shape[0] * test_vectors.shape[1])))

UNIQUE_LABELS_N = len(set(train_source['target']))
print('Количество уникальных меток', UNIQUE_LABELS_N)
set(train_source['target'])

train_dataset = SparseFeaturesDataset(train_vectors, train_source['target'])
test_dataset = SparseFeaturesDataset(test_vectors, test_source['target'])

model = torch.nn.Sequential(nn.Linear(UNIQUE_WORDS_N, HIDDEN_NEURONS),
                            torch.nn.Dropout(DROPOUT_p),
                            torch.nn.ReLU(),
                            nn.Linear(HIDDEN_NEURONS, UNIQUE_LABELS_N))

scheduler = lambda optim: \
    torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=SCHED_PAT, factor=0.5, verbose=True)

histories[experiment_name], best_models[experiment_name] = train_eval_loop(
                                            model=model,
                                            train_dataset=train_dataset,
                                            val_dataset=test_dataset,
                                            criterion=F.cross_entropy,
                                            lr=LR,
                                            epoch_n=200,
                                            batch_size=BATCH_SIZE,
                                            l2_reg_alpha=L2REG,
                                            lr_scheduler_ctor=scheduler,
                                            best_acc_type = 'acc',
                                            test_dataset=test_dataset,
                                            early_stopping_patience=6,
                                            experiment_name = experiment_name)

In [ ]:
min_token_size=2
ADD_STEMMS = True
MIN_COUNT = 2
VECTORIZATION_MODE = 'tflidf'

if 'pmi' in VECTORIZATION_MODE:
    MAX_QUAN = 0.8
    MAX_TYPE = 'MAX_QUAN={}'.format(MAX_QUAN) #only for generate experiment_name
else:
    MAX_DF = 0.8
    MAX_TYPE = 'MAX_DF={}'.format(MAX_DF) #only for generate experiment_name

HIDDEN_NEURONS = 240
DROPOUT_p = 0.4
BATCH_SIZE = 120
LR = 1e-2
L2REG = 0
SCHED_PAT = 4

experiment_name = 'mtoken_size={}_MIN_COUNT={}_ADD_STEMMS_{}_{}_NN_Linear={}_Dropout={}_lr={}_BATCH={}_L2REG={}_SCHED_PAT={}'.format(
                    min_token_size, MIN_COUNT, 
                    MAX_TYPE,
                    VECTORIZATION_MODE, HIDDEN_NEURONS, DROPOUT_p, LR, BATCH_SIZE, L2REG, SCHED_PAT)


train_tokenized = tokenize_corpus(train_source['data'], min_token_size=2)
test_tokenized = tokenize_corpus(test_source['data'], min_token_size=2)

print(' '.join(train_tokenized[0]))

if ADD_STEMMS:
    #PorterStemmer:
    from nltk.stem.porter import PorterStemmer
    ps = PorterStemmer()
    converter = ps.stem

    train_tokenized = tokenize_corpus_convert(train_tokenized, converter=converter, addition = True)
    test_tokenized = tokenize_corpus_convert(test_tokenized, converter=converter, addition = True)

    print(' '.join(train_tokenized[0]))

    
if 'pmi' in VECTORIZATION_MODE:
    vocabulary, word_doc_pmi = build_vocabulary_pmi(train_tokenized, train_source['target'], 
                                                invers_quantile_threshold = MAX_QUAN, min_count = MIN_COUNT)
    
    train_vectors = vectorize_texts(train_tokenized, vocabulary, word_doc_pmi, mode=VECTORIZATION_MODE)
    test_vectors = vectorize_texts(test_tokenized, vocabulary, word_doc_pmi, mode=VECTORIZATION_MODE)
else:
    vocabulary, word_doc_freq = build_vocabulary(train_tokenized, max_doc_freq=MAX_DF, min_count=MIN_COUNT)
    
    train_vectors = vectorize_texts(train_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)
    test_vectors = vectorize_texts(test_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)



UNIQUE_WORDS_N = len(vocabulary)
print('Количество уникальных токенов', UNIQUE_WORDS_N)
print(list(vocabulary.items())[:10])

print('Размерность матрицы признаков обучающей выборки', train_vectors.shape)
print('Размерность матрицы признаков тестовой выборки', test_vectors.shape)
print()
print('Количество ненулевых элементов в обучающей выборке', train_vectors.nnz)
print('Процент заполненности матрицы признаков {:.2f}%'.format(train_vectors.nnz * 100 / (train_vectors.shape[0] * train_vectors.shape[1])))
print()
print('Количество ненулевых элементов в тестовой выборке', test_vectors.nnz)
print('Процент заполненности матрицы признаков {:.2f}%'.format(test_vectors.nnz * 100 / (test_vectors.shape[0] * test_vectors.shape[1])))

UNIQUE_LABELS_N = len(set(train_source['target']))
print('Количество уникальных меток', UNIQUE_LABELS_N)
set(train_source['target'])

train_dataset = SparseFeaturesDataset(train_vectors, train_source['target'])
test_dataset = SparseFeaturesDataset(test_vectors, test_source['target'])

model = torch.nn.Sequential(nn.Linear(UNIQUE_WORDS_N, HIDDEN_NEURONS),
                            torch.nn.Dropout(DROPOUT_p),
                            torch.nn.ReLU(),
                            nn.Linear(HIDDEN_NEURONS, UNIQUE_LABELS_N))

scheduler = lambda optim: \
    torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=SCHED_PAT, factor=0.5, verbose=True)

histories[experiment_name], best_models[experiment_name] = train_eval_loop(
                                            model=model,
                                            train_dataset=train_dataset,
                                            val_dataset=test_dataset,
                                            criterion=F.cross_entropy,
                                            lr=LR,
                                            epoch_n=200,
                                            batch_size=BATCH_SIZE,
                                            l2_reg_alpha=L2REG,
                                            lr_scheduler_ctor=scheduler,
                                            best_acc_type = 'acc',
                                            test_dataset=test_dataset,
                                            early_stopping_patience=6,
                                            experiment_name = experiment_name)

In [ ]:
min_token_size=2
ADD_STEMMS = True
MIN_COUNT = 2
VECTORIZATION_MODE = 'tflidf'

if 'pmi' in VECTORIZATION_MODE:
    MAX_QUAN = 0.8
    MAX_TYPE = 'MAX_QUAN={}'.format(MAX_QUAN) #only for generate experiment_name
else:
    MAX_DF = 0.8
    MAX_TYPE = 'MAX_DF={}'.format(MAX_DF) #only for generate experiment_name

HIDDEN_NEURONS = 240
DROPOUT_p = 0.4
BATCH_SIZE = 64
LR = 1e-2
L2REG = 1e-6
SCHED_PAT = 4

experiment_name = 'mtoken_size={}_MIN_COUNT={}_ADD_STEMMS_{}_{}_NN_Linear={}_Dropout={}_lr={}_BATCH={}_L2REG={}_SCHED_PAT={}'.format(
                    min_token_size, MIN_COUNT, 
                    MAX_TYPE,
                    VECTORIZATION_MODE, HIDDEN_NEURONS, DROPOUT_p, LR, BATCH_SIZE, L2REG, SCHED_PAT)


train_tokenized = tokenize_corpus(train_source['data'], min_token_size=2)
test_tokenized = tokenize_corpus(test_source['data'], min_token_size=2)

print(' '.join(train_tokenized[0]))

if ADD_STEMMS:
    #PorterStemmer:
    from nltk.stem.porter import PorterStemmer
    ps = PorterStemmer()
    converter = ps.stem

    train_tokenized = tokenize_corpus_convert(train_tokenized, converter=converter, addition = True)
    test_tokenized = tokenize_corpus_convert(test_tokenized, converter=converter, addition = True)

    print(' '.join(train_tokenized[0]))

    
if 'pmi' in VECTORIZATION_MODE:
    vocabulary, word_doc_pmi = build_vocabulary_pmi(train_tokenized, train_source['target'], 
                                                invers_quantile_threshold = MAX_QUAN, min_count = MIN_COUNT)
    
    train_vectors = vectorize_texts(train_tokenized, vocabulary, word_doc_pmi, mode=VECTORIZATION_MODE)
    test_vectors = vectorize_texts(test_tokenized, vocabulary, word_doc_pmi, mode=VECTORIZATION_MODE)
else:
    vocabulary, word_doc_freq = build_vocabulary(train_tokenized, max_doc_freq=MAX_DF, min_count=MIN_COUNT)
    
    train_vectors = vectorize_texts(train_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)
    test_vectors = vectorize_texts(test_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)



UNIQUE_WORDS_N = len(vocabulary)
print('Количество уникальных токенов', UNIQUE_WORDS_N)
print(list(vocabulary.items())[:10])

print('Размерность матрицы признаков обучающей выборки', train_vectors.shape)
print('Размерность матрицы признаков тестовой выборки', test_vectors.shape)
print()
print('Количество ненулевых элементов в обучающей выборке', train_vectors.nnz)
print('Процент заполненности матрицы признаков {:.2f}%'.format(train_vectors.nnz * 100 / (train_vectors.shape[0] * train_vectors.shape[1])))
print()
print('Количество ненулевых элементов в тестовой выборке', test_vectors.nnz)
print('Процент заполненности матрицы признаков {:.2f}%'.format(test_vectors.nnz * 100 / (test_vectors.shape[0] * test_vectors.shape[1])))

UNIQUE_LABELS_N = len(set(train_source['target']))
print('Количество уникальных меток', UNIQUE_LABELS_N)
set(train_source['target'])

train_dataset = SparseFeaturesDataset(train_vectors, train_source['target'])
test_dataset = SparseFeaturesDataset(test_vectors, test_source['target'])

model = torch.nn.Sequential(nn.Linear(UNIQUE_WORDS_N, HIDDEN_NEURONS),
                            torch.nn.Dropout(DROPOUT_p),
                            torch.nn.ReLU(),
                            nn.Linear(HIDDEN_NEURONS, UNIQUE_LABELS_N))

scheduler = lambda optim: \
    torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=SCHED_PAT, factor=0.5, verbose=True)

histories[experiment_name], best_models[experiment_name] = train_eval_loop(
                                            model=model,
                                            train_dataset=train_dataset,
                                            val_dataset=test_dataset,
                                            criterion=F.cross_entropy,
                                            lr=LR,
                                            epoch_n=200,
                                            batch_size=BATCH_SIZE,
                                            l2_reg_alpha=L2REG,
                                            lr_scheduler_ctor=scheduler,
                                            best_acc_type = 'acc',
                                            test_dataset=test_dataset,
                                            early_stopping_patience=6,
                                            experiment_name = experiment_name)

In [ ]:
min_token_size=2
ADD_STEMMS = True
MIN_COUNT = 2
VECTORIZATION_MODE = 'tflidf'

if 'pmi' in VECTORIZATION_MODE:
    MAX_QUAN = 0.8
    MAX_TYPE = 'MAX_QUAN={}'.format(MAX_QUAN) #only for generate experiment_name
else:
    MAX_DF = 0.8
    MAX_TYPE = 'MAX_DF={}'.format(MAX_DF) #only for generate experiment_name

HIDDEN_NEURONS = 240
DROPOUT_p = 0.4
BATCH_SIZE = 64
LR = 1e-2
L2REG = 0
SCHED_PAT = 6

experiment_name = 'mtoken_size={}_MIN_COUNT={}_ADD_STEMMS_{}_{}_NN_Linear={}_Dropout={}_lr={}_BATCH={}_L2REG={}_SCHED_PAT={}'.format(
                    min_token_size, MIN_COUNT, 
                    MAX_TYPE,
                    VECTORIZATION_MODE, HIDDEN_NEURONS, DROPOUT_p, LR, BATCH_SIZE, L2REG, SCHED_PAT)


train_tokenized = tokenize_corpus(train_source['data'], min_token_size=2)
test_tokenized = tokenize_corpus(test_source['data'], min_token_size=2)

print(' '.join(train_tokenized[0]))

if ADD_STEMMS:
    #PorterStemmer:
    from nltk.stem.porter import PorterStemmer
    ps = PorterStemmer()
    converter = ps.stem

    train_tokenized = tokenize_corpus_convert(train_tokenized, converter=converter, addition = True)
    test_tokenized = tokenize_corpus_convert(test_tokenized, converter=converter, addition = True)

    print(' '.join(train_tokenized[0]))

    
if 'pmi' in VECTORIZATION_MODE:
    vocabulary, word_doc_pmi = build_vocabulary_pmi(train_tokenized, train_source['target'], 
                                                invers_quantile_threshold = MAX_QUAN, min_count = MIN_COUNT)
    
    train_vectors = vectorize_texts(train_tokenized, vocabulary, word_doc_pmi, mode=VECTORIZATION_MODE)
    test_vectors = vectorize_texts(test_tokenized, vocabulary, word_doc_pmi, mode=VECTORIZATION_MODE)
else:
    vocabulary, word_doc_freq = build_vocabulary(train_tokenized, max_doc_freq=MAX_DF, min_count=MIN_COUNT)
    
    train_vectors = vectorize_texts(train_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)
    test_vectors = vectorize_texts(test_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)



UNIQUE_WORDS_N = len(vocabulary)
print('Количество уникальных токенов', UNIQUE_WORDS_N)
print(list(vocabulary.items())[:10])

print('Размерность матрицы признаков обучающей выборки', train_vectors.shape)
print('Размерность матрицы признаков тестовой выборки', test_vectors.shape)
print()
print('Количество ненулевых элементов в обучающей выборке', train_vectors.nnz)
print('Процент заполненности матрицы признаков {:.2f}%'.format(train_vectors.nnz * 100 / (train_vectors.shape[0] * train_vectors.shape[1])))
print()
print('Количество ненулевых элементов в тестовой выборке', test_vectors.nnz)
print('Процент заполненности матрицы признаков {:.2f}%'.format(test_vectors.nnz * 100 / (test_vectors.shape[0] * test_vectors.shape[1])))

UNIQUE_LABELS_N = len(set(train_source['target']))
print('Количество уникальных меток', UNIQUE_LABELS_N)
set(train_source['target'])

train_dataset = SparseFeaturesDataset(train_vectors, train_source['target'])
test_dataset = SparseFeaturesDataset(test_vectors, test_source['target'])

model = torch.nn.Sequential(nn.Linear(UNIQUE_WORDS_N, HIDDEN_NEURONS),
                            torch.nn.Dropout(DROPOUT_p),
                            torch.nn.ReLU(),
                            nn.Linear(HIDDEN_NEURONS, UNIQUE_LABELS_N))

scheduler = lambda optim: \
    torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=SCHED_PAT, factor=0.5, verbose=True)

histories[experiment_name], best_models[experiment_name] = train_eval_loop(
                                            model=model,
                                            train_dataset=train_dataset,
                                            val_dataset=test_dataset,
                                            criterion=F.cross_entropy,
                                            lr=LR,
                                            epoch_n=200,
                                            batch_size=BATCH_SIZE,
                                            l2_reg_alpha=L2REG,
                                            lr_scheduler_ctor=scheduler,
                                            best_acc_type = 'acc',
                                            test_dataset=test_dataset,
                                            early_stopping_patience=6,
                                            experiment_name = experiment_name)

In [ ]:
min_token_size=1
ADD_STEMMS = True
MIN_COUNT = 2
VECTORIZATION_MODE = 'tflidf'

if 'pmi' in VECTORIZATION_MODE:
    MAX_QUAN = 0.8
    MAX_TYPE = 'MAX_QUAN={}'.format(MAX_QUAN) #only for generate experiment_name
else:
    MAX_DF = 0.8
    MAX_TYPE = 'MAX_DF={}'.format(MAX_DF) #only for generate experiment_name

HIDDEN_NEURONS = 240
DROPOUT_p = 0.4
BATCH_SIZE = 64
LR = 1e-2
L2REG = 0
SCHED_PAT = 4

experiment_name = 'mtoken_size={}_MIN_COUNT={}_ADD_STEMMS_{}_{}_NN_Linear={}_Dropout={}_lr={}_BATCH={}_L2REG={}_SCHED_PAT={}'.format(
                    min_token_size, MIN_COUNT, 
                    MAX_TYPE,
                    VECTORIZATION_MODE, HIDDEN_NEURONS, DROPOUT_p, LR, BATCH_SIZE, L2REG, SCHED_PAT)


train_tokenized = tokenize_corpus(train_source['data'], min_token_size=2)
test_tokenized = tokenize_corpus(test_source['data'], min_token_size=2)

print(' '.join(train_tokenized[0]))

if ADD_STEMMS:
    #PorterStemmer:
    from nltk.stem.porter import PorterStemmer
    ps = PorterStemmer()
    converter = ps.stem

    train_tokenized = tokenize_corpus_convert(train_tokenized, converter=converter, addition = True)
    test_tokenized = tokenize_corpus_convert(test_tokenized, converter=converter, addition = True)

    print(' '.join(train_tokenized[0]))

    
if 'pmi' in VECTORIZATION_MODE:
    vocabulary, word_doc_pmi = build_vocabulary_pmi(train_tokenized, train_source['target'], 
                                                invers_quantile_threshold = MAX_QUAN, min_count = MIN_COUNT)
    
    train_vectors = vectorize_texts(train_tokenized, vocabulary, word_doc_pmi, mode=VECTORIZATION_MODE)
    test_vectors = vectorize_texts(test_tokenized, vocabulary, word_doc_pmi, mode=VECTORIZATION_MODE)
else:
    vocabulary, word_doc_freq = build_vocabulary(train_tokenized, max_doc_freq=MAX_DF, min_count=MIN_COUNT)
    
    train_vectors = vectorize_texts(train_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)
    test_vectors = vectorize_texts(test_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)



UNIQUE_WORDS_N = len(vocabulary)
print('Количество уникальных токенов', UNIQUE_WORDS_N)
print(list(vocabulary.items())[:10])

print('Размерность матрицы признаков обучающей выборки', train_vectors.shape)
print('Размерность матрицы признаков тестовой выборки', test_vectors.shape)
print()
print('Количество ненулевых элементов в обучающей выборке', train_vectors.nnz)
print('Процент заполненности матрицы признаков {:.2f}%'.format(train_vectors.nnz * 100 / (train_vectors.shape[0] * train_vectors.shape[1])))
print()
print('Количество ненулевых элементов в тестовой выборке', test_vectors.nnz)
print('Процент заполненности матрицы признаков {:.2f}%'.format(test_vectors.nnz * 100 / (test_vectors.shape[0] * test_vectors.shape[1])))

UNIQUE_LABELS_N = len(set(train_source['target']))
print('Количество уникальных меток', UNIQUE_LABELS_N)
set(train_source['target'])

train_dataset = SparseFeaturesDataset(train_vectors, train_source['target'])
test_dataset = SparseFeaturesDataset(test_vectors, test_source['target'])

model = torch.nn.Sequential(nn.Linear(UNIQUE_WORDS_N, HIDDEN_NEURONS),
                            torch.nn.Dropout(DROPOUT_p),
                            torch.nn.ReLU(),
                            nn.Linear(HIDDEN_NEURONS, UNIQUE_LABELS_N))

scheduler = lambda optim: \
    torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=SCHED_PAT, factor=0.5, verbose=True)

histories[experiment_name], best_models[experiment_name] = train_eval_loop(
                                            model=model,
                                            train_dataset=train_dataset,
                                            val_dataset=test_dataset,
                                            criterion=F.cross_entropy,
                                            lr=LR,
                                            epoch_n=200,
                                            batch_size=BATCH_SIZE,
                                            l2_reg_alpha=L2REG,
                                            lr_scheduler_ctor=scheduler,
                                            best_acc_type = 'acc',
                                            test_dataset=test_dataset,
                                            early_stopping_patience=6,
                                            experiment_name = experiment_name)

In [ ]:
# min_token_size=2
# ADD_STEMMS = True
# MIN_COUNT = 2
# VECTORIZATION_MODE = 'tfpmi'

# if 'pmi' in VECTORIZATION_MODE:
#     MAX_QUAN = 0.98
#     MAX_TYPE = 'MAX_QUAN={}'.format(MAX_QUAN) #only for generate experiment_name
# else:
#     MAX_DF = 0.8
#     MAX_TYPE = 'MAX_DF={}'.format(MAX_DF) #only for generate experiment_name

# HIDDEN_NEURONS = 240
# DROPOUT_p = 0.4
# BATCH_SIZE = 32
# LR = 1e-2
# L2REG = 0
# SCHED_PAT = 4

# experiment_name = 'mtoken_size={}_MIN_COUNT={}_ADD_STEMMS_{}_{}_NN_Linear={}_Dropout={}_lr={}_BATCH={}_L2REG={}_SCHED_PAT={}'.format(
#                     min_token_size, MIN_COUNT, 
#                     MAX_TYPE,
#                     VECTORIZATION_MODE, HIDDEN_NEURONS, DROPOUT_p, LR, BATCH_SIZE, L2REG, SCHED_PAT)


# train_tokenized = tokenize_corpus(train_source['data'], min_token_size=2)
# test_tokenized = tokenize_corpus(test_source['data'], min_token_size=2)

# print(' '.join(train_tokenized[0]))

# if ADD_STEMMS:
#     #PorterStemmer:
#     from nltk.stem.porter import PorterStemmer
#     ps = PorterStemmer()
#     converter = ps.stem

#     train_tokenized = tokenize_corpus_convert(train_tokenized, converter=converter, addition = True)
#     test_tokenized = tokenize_corpus_convert(test_tokenized, converter=converter, addition = True)

#     print(' '.join(train_tokenized[0]))

    
# if 'pmi' in VECTORIZATION_MODE:
#     vocabulary, word_doc_pmi = build_vocabulary_pmi(train_tokenized, train_source['target'], 
#                                                 invers_quantile_threshold = MAX_QUAN, min_count = MIN_COUNT)
    
#     train_vectors = vectorize_texts(train_tokenized, vocabulary, word_doc_pmi, mode=VECTORIZATION_MODE)
#     test_vectors = vectorize_texts(test_tokenized, vocabulary, word_doc_pmi, mode=VECTORIZATION_MODE)
# else:
#     vocabulary, word_doc_freq = build_vocabulary(train_tokenized, max_doc_freq=MAX_DF, min_count=MIN_COUNT)
    
#     train_vectors = vectorize_texts(train_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)
#     test_vectors = vectorize_texts(test_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)



# UNIQUE_WORDS_N = len(vocabulary)
# print('Количество уникальных токенов', UNIQUE_WORDS_N)
# print(list(vocabulary.items())[:10])

# print('Размерность матрицы признаков обучающей выборки', train_vectors.shape)
# print('Размерность матрицы признаков тестовой выборки', test_vectors.shape)
# print()
# print('Количество ненулевых элементов в обучающей выборке', train_vectors.nnz)
# print('Процент заполненности матрицы признаков {:.2f}%'.format(train_vectors.nnz * 100 / (train_vectors.shape[0] * train_vectors.shape[1])))
# print()
# print('Количество ненулевых элементов в тестовой выборке', test_vectors.nnz)
# print('Процент заполненности матрицы признаков {:.2f}%'.format(test_vectors.nnz * 100 / (test_vectors.shape[0] * test_vectors.shape[1])))

# UNIQUE_LABELS_N = len(set(train_source['target']))
# print('Количество уникальных меток', UNIQUE_LABELS_N)
# set(train_source['target'])

# train_dataset = SparseFeaturesDataset(train_vectors, train_source['target'])
# test_dataset = SparseFeaturesDataset(test_vectors, test_source['target'])

# model = torch.nn.Sequential(nn.Linear(UNIQUE_WORDS_N, HIDDEN_NEURONS),
#                             torch.nn.Dropout(DROPOUT_p),
#                             torch.nn.ReLU(),
#                             nn.Linear(HIDDEN_NEURONS, UNIQUE_LABELS_N))

# scheduler = lambda optim: \
#     torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=SCHED_PAT, factor=0.5, verbose=True)

# histories[experiment_name], best_models[experiment_name] = train_eval_loop(
#                                             model=model,
#                                             train_dataset=train_dataset,
#                                             val_dataset=test_dataset,
#                                             criterion=F.cross_entropy,
#                                             lr=LR,
#                                             epoch_n=200,
#                                             batch_size=BATCH_SIZE,
#                                             l2_reg_alpha=L2REG,
#                                             lr_scheduler_ctor=scheduler,
#                                             best_acc_type = 'acc',
#                                             test_dataset=test_dataset,
#                                             early_stopping_patience=6,
#                                             experiment_name = experiment_name)

In [ ]:
show_experiments_stats(histories, show_plots = True, only_BEST_MODEL_CALC = True)